In [512]:
import numpy as np
import pandas as pd
import re
import copy
import xgboost
from google_dict import *
from nltk.corpus import wordnet as wn
import difflib
import time
import os
from IRDM_Functions import *
from replace_brand_dict import *
stoplist_wo_can=stoplist[:]
stoplist_wo_can.remove('can')
from config import *
from time import time
import csv
import os
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
stoplist = stopwords.words('english')
stoplist.append('till')  # add 'till' to stoplist
import warnings
warnings.filterwarnings('ignore')

In [335]:
train_df = pd.read_csv('/home/ubuntu/web-datamining/IRDM/Data/train.csv',encoding = "ISO-8859-1")
test_df = pd.read_csv('/home/ubuntu/web-datamining/IRDM/Data/test.csv',encoding = "ISO-8859-1")
attributes = pd.read_csv('/home/ubuntu/web-datamining/IRDM/Data/attributes.csv', encoding="ISO-8859-1")
pro_descr = pd.read_csv('/home/ubuntu/web-datamining/IRDM/Data/product_descriptions.csv')

In [167]:
attributes = attributes.dropna(axis=0,how='all')

# Extract data from Description and Attribute dataset

In [168]:
All_df = pd.concat((train_df,test_df),axis=0,ignore_index=True)
All_df = pd.merge(All_df,pro_descr,how='left',on='product_uid')

In [169]:
brand_df = attributes[attributes['name']=='MFG Brand Name']
brand_df = brand_df.rename(columns={'value':'Brand'})

In [170]:
All_df = pd.merge(All_df, brand_df, how='left', on='product_uid')
All_df = All_df.drop('name',axis=1)

In [171]:
All_df.Brand.fillna('none',inplace=True)

In [172]:
####### need to exploration 
np.any(All_df['relevance'].isnull()==True)

True

In [147]:
### extract Materials from attributes
material_df = attributes[attributes['name'].str.contains('Material')==True]

dup_material = material_df[material_df.duplicated('product_uid')==True]
nodup_material = material_df[material_df.duplicated('product_uid')==False]
dup_material.fillna('none',inplace=True)
nodup_material.fillna('none',inplace=True)

dm = dup_material.index
material_df = material_df.drop(dm)
for i in dm:
    material_df['value'][material_df['product_uid']==dup_material.ix[i,'product_uid']]=\
    material_df['value'][material_df['product_uid']==dup_material.ix[i,'product_uid']]+','+\
    dup_material.ix[i,'value']

In [148]:
# material_df.to_csv('Material_cloud_1.csv')

In [1]:
### extract Color from attribute dataset
color_df = attributes[attributes['name'].str.contains('Color')==True]
color_df.fillna('none',inplace=True)

In [35]:
dup_color = color_df[color_df.duplicated('product_uid')==True]
nodup_color = color_df[color_df.duplicated('product_uid')==False]

In [36]:
dc = dup_color.index
color_df = color_df.drop(dc)

In [37]:
for j in dc:
    color_df['value'][color_df['product_uid']==dup_color.ix[j,'product_uid']]=\
    color_df['value'][color_df['product_uid']==dup_color.ix[j,'product_uid']]+','+\
    dup_color.ix[j,'value']

In [38]:
# color_df.to_csv('Color_cloud.csv')

In [99]:
### extract bullet from attributes
bullet_df = attributes[attributes['name'].str.contains('Bullet')==True]

In [100]:
dup_bullet = bullet_df[bullet_df.duplicated('product_uid')==True]
db = dup_bullet.index
bullet_df_copy = copy.deepcopy(bullet_df)
bullet_df_copy = bullet_df_copy.drop(db)

In [116]:
bullet_df = bullet_df.dropna()
for i in list(set(list(attributes['product_uid']))):
    bullet_df_copy['value'][bullet_df_copy['product_uid']==i] ='.'.join(list(bullet_df['value'][bullet_df['product_uid']==i]))
    

In [117]:
bullet_df_copy.to_csv('Bullet_cloud.csv')

In [176]:
bdi = pd.Series(list(brand_df.index))
mdd = pd.Series(list(attributes[attributes['name'].str.contains('Material')==True].index))
cdd = pd.Series(list(attributes[attributes['name'].str.contains('Color')==True].index))
bdd = pd.Series(list(attributes[attributes['name'].str.contains('Bullet')==True].index))

In [177]:
index_to_drop = pd.concat((bdi,mdd,cdd,bdd),axis=0)
attributes = attributes.drop(set(list(index_to_drop)))

In [178]:
### extract product height
Height_family = attributes[attributes['name'].str.contains('Product Height')]

In [179]:
Height_family['name'].value_counts()

Product Height (in.)            54698
Actual Product Height (in.)       265
Nominal Product Height (In.)      179
Product Height (ft.)               83
Maximum Product Height (in.)       66
Minimum Product Height (In.)       48
Name: name, dtype: int64

In [180]:
# for i in range(len(list(Height_family['name'].groupby(Height_family['product_uid'])))):
#     if len(list(Height_family['name'].groupby(Height_family['product_uid']))[i])>2:
#         print(list(Height_family['name'].groupby(Height_family['product_uid']))[i])

In [181]:
dup_Height_family = Height_family[Height_family.duplicated('product_uid')==True]
Height_family = Height_family.drop(dup_Height_family.index)
collect_id = []
for i in dup_Height_family['product_uid']:
    j = dup_Height_family[dup_Height_family['product_uid']==i].index
    for n in np.arange(0,len(j)):
        m = j.tolist()[n]
        if dup_Height_family.ix[m,'name']=='Product Height (in.)':
            Height_family['value'][Height_family['product_uid']==i] = dup_Height_family.ix[m,'value']
            if n==len(j)-1:
                dup_Height_family = dup_Height_family.drop(j)
            else:
                collect_id.append(j)
            

Height_family['value'][Height_family['name']=='Product Height (ft.)']=\
Height_family['value'][Height_family['name']=='Product Height (ft.)']*12

In [182]:
Height_family['name'].value_counts()

Product Height (in.)            54380
Actual Product Height (in.)       265
Nominal Product Height (In.)      179
Product Height (ft.)               83
Maximum Product Height (in.)       66
Name: name, dtype: int64

In [183]:
### extract product width
Width_family = attributes[attributes['name'].str.contains('Product Width')]

In [184]:
Width_family['name'].value_counts()

Product Width (in.)           61137
Actual Product Width (in.)      265
Product Width (ft.)             156
Name: name, dtype: int64

In [185]:
dup_Width_family = Width_family[Width_family.duplicated('product_uid')==True]
Width_family = Width_family.drop(dup_Width_family.index)
for i in dup_Width_family['product_uid']:
    j = dup_Width_family[dup_Width_family['product_uid']==i].index
    for n in np.arange(0,len(j)):
        m = j.tolist()[n]
        if dup_Width_family.ix[m,'name']=='Product Width (in.)':
            Width_family['value'][Width_family['product_uid']==i] = dup_Width_family.ix[m,'value']
            if n==len(j)-1:
                dup_Width_family = dup_Width_family.drop(j)
            else:
                collect_id.append(j)
            
        

Width_family['value'][Width_family['name']=='Product Width (ft.)']=\
Width_family['value'][Width_family['name']=='Product Width (ft.)']*12

In [186]:
### extract product depth
Depth_family = attributes[attributes['name'].str.contains('Product Depth')]

In [187]:
Depth_family['name'].value_counts()

Product Depth (in.)           53652
Actual Product Depth (in.)      265
Name: name, dtype: int64

In [188]:
### extract product weight
Weight_family = attributes[attributes['name'].str.contains('Product Weight')]

In [189]:
Weight_family['name'].value_counts()

Product Weight (lb.)          45175
Product Weight (oz.)            704
Total Product Weight (lb.)       35
Name: name, dtype: int64

In [190]:
dup_Weight_family = Weight_family[Weight_family.duplicated('product_uid')==True]

In [191]:
Weight_family = Weight_family.drop(dup_Weight_family.index)

In [192]:
### extract product Length
Length_family = attributes[attributes['name'].str.contains('Product Length')]

In [193]:
Length_family['name'].value_counts()

Product Length (in.)            16705
Product Length (ft.)             2241
Nominal Product Length (ft.)      296
Nominal Product Length (in.)      259
Maximum Product Length (in.)      202
Minimum Product Length (in.)       32
Name: name, dtype: int64

In [194]:
dup_Length_family = Length_family[Length_family.duplicated('product_uid')==True]
Length_family = Length_family.drop(dup_Length_family.index)
for i in dup_Length_family['product_uid']:
    j = dup_Length_family[dup_Length_family['product_uid']==i].index
    for n in np.arange(0,len(j)):
        m = j.tolist()[n]
        if dup_Length_family.ix[m,'name']=='Product Length (in.)':
            Length_family['value'][Length_family['product_uid']==i] = dup_Length_family.ix[m,'value']
            Length_family['name'][Length_family['product_uid']==i] = 'Product Length (in.)'
            if n==len(j)-1:
                dup_Length_family = dup_Length_family.drop(j)
            else:
                collect_id.append(j)       

In [195]:
dup_Length_family['name'].value_counts()

Product Length (ft.)            54
Nominal Product Length (in.)    25
Minimum Product Length (in.)    20
Name: name, dtype: int64

In [196]:
Length_ft = dup_Length_family[dup_Length_family['name']=='Product Length (ft.)']
for i in Length_ft['product_uid']:
    j = Length_ft[Length_ft['product_uid']==i].index.tolist()[0]
    k = Length_family[Length_family['product_uid']==i].index.tolist()[0]
    if Length_family.ix[k,'name']=='Product Length (in.)':
        dup_Length_family = dup_Length_family.drop(dup_Length_family[dup_Length_family['product_uid']==i].index)   
    if Length_family.ix[k,'name']!='Product Length (in.)':
        Length_family.ix[k,'name']='Product Length (in.)'
        Length_family.ix[k,'value']=Length_ft.ix[j,'value']*12
        dup_Length_family = dup_Length_family.drop(dup_Length_family[dup_Length_family['product_uid']==i].index)   

In [197]:
Length_family['value'][Length_family['name']=='Product Length (ft.)']=Length_family['value'][Length_family['name']=='Product Length (ft.)']*12
Length_family['name'][Length_family['name']=='Product Length (ft.)']='Product Length (in.)'

In [198]:
### extract other attributes
# CER_LIS = attributes[attributes['name'].str.contains('Certifications and Listings')]
I_O = attributes[attributes['name'].str.contains('Indoor/Outdoor')]
C_R = attributes[attributes['name'].str.contains('Commercial / Residential')]
ESC = attributes[attributes['name'].str.contains('ENERGY STAR Certified')]
P_Q = attributes[attributes['name'].str.contains('Package Quantity')]
H_I = attributes[attributes['name'].str.contains('Hardware Included')]
dup_HI_family = H_I[H_I.duplicated('product_uid')==True]
H_I = H_I.drop(dup_HI_family.index)

In [199]:
features_to_drop = ['Unnamed: 0','name']
material_df = material_df.drop(features_to_drop,axis=1)
material_df = material_df.rename(columns={'value':'Material'})
color_df = color_df.drop(features_to_drop,axis=1)
color_df = color_df.rename(columns={'value':'Color'})
bullet_df_1 = bullet_df_copy.drop(features_to_drop,axis=1)
bullet_df_1 = bullet_df_1.rename(columns={'value':'Bullet'})
Height_family = Height_family.drop('name',axis=1)
Height_family = Height_family.rename(columns={'value':'Product Height (in.)'})
Width_family = Width_family.drop('name',axis=1)
Width_family = Width_family.rename(columns={'value':'Product Width (in.)'})
Depth_family = Depth_family.drop('name',axis=1)
Depth_family = Depth_family.rename(columns={'value':'Product Depth (in.)'})
Weight_family = Weight_family.drop('name',axis=1)
Weight_family = Weight_family.rename(columns={'value':'Product Weight (lb.)'})
Length_family = Length_family.drop('name',axis=1)
Length_family = Length_family.rename(columns={'value':'Product Length (in.)'})
I_O = I_O.drop('name',axis=1)
I_O = I_O.rename(columns={'value':'Indoor/Outdoor'})
C_R = C_R.drop('name',axis=1)
C_R = C_R.rename(columns={'value':'Commercial / Residential'})
ESC = ESC.drop('name',axis=1)
ESC = ESC.rename(columns={'value':'ENERGY STAR Certified'})
P_Q = P_Q.drop('name',axis=1)
P_Q = P_Q.rename(columns={'value':'Package Quantity'})
H_I = H_I.drop('name',axis=1)
H_I = H_I.rename(columns={'value':'Hardware Included'})

In [200]:
List_att = attributes[attributes['name'].str.contains('List')]
List_att = List_att[~List_att['value'].str.contains('No')]
L = List_att['product_uid'].unique()
Certi_att = attributes[attributes['name'].str.contains('Certifi')]
Certi_att = Certi_att[~attributes['name'].str.contains('No')]
C = Certi_att['product_uid'].unique()
CER_LIS = np.concatenate((C,L), axis=0)
CER_LIS = list(set(CER_LIS))

In [201]:
All_df['Certification'] = 0
for i in CER_LIS:
    All_df['Certification'][All_df['product_uid']==i]=1

In [202]:
All_df = pd.merge(All_df, material_df, how='left', on='product_uid')
All_df = pd.merge(All_df, color_df, how='left', on='product_uid')
All_df = pd.merge(All_df, bullet_df_1, how='left', on='product_uid')
All_df = pd.merge(All_df, Height_family, how='left', on='product_uid')
All_df = pd.merge(All_df, Width_family, how='left', on='product_uid')
All_df = pd.merge(All_df, Depth_family, how='left', on='product_uid')
All_df = pd.merge(All_df, Weight_family, how='left', on='product_uid')
All_df = pd.merge(All_df, Length_family, how='left', on='product_uid')
All_df = pd.merge(All_df, I_O, how='left', on='product_uid')
All_df = pd.merge(All_df, C_R, how='left', on='product_uid')
All_df = pd.merge(All_df, ESC, how='left', on='product_uid')
All_df = pd.merge(All_df, P_Q, how='left', on='product_uid')
All_df = pd.merge(All_df, H_I, how='left', on='product_uid')

# Text processing

### functions

In [204]:
def simple_parser(s):
    s = re.sub('&amp;', '&', s)
    s = re.sub('&nbsp;', '', s)
    s = re.sub('&#39;', '', s)
    s = s.replace("-"," ")
    s = s.replace("+"," ")
    s = re.sub(r'(?<=[a-zA-Z])\/(?=[a-zA-Z])', ' ', s)
    s = re.sub(r'(?<=\))(?=[a-zA-Z0-9])', ' ', s) # add space between parentheses and letters
    s = re.sub(r'(?<=[a-zA-Z0-9])(?=\()', ' ', s) # add space between parentheses and letters
    s = re.sub(r'(?<=[a-zA-Z][\.\,])(?=[a-zA-Z])', ' ', s) # add space after dot or colon between letters
    s = re.sub('[^a-zA-Z0-9\n\ ]', '', s)
    return s

In [205]:
def is_word_in_string(word,s):
    return word in s.split() 
    
def create_bigrams(s):
    lst=[word for word in s.split() if len(re.sub('[^0-9]', '', word))==0 and len(word)>2]
    output=""
    i=0
    if len(lst)>=2:
        while i<len(lst)-1:
            output+= " "+lst[i]+"_"+lst[i+1]
            i+=1
    return output

In [206]:
def seq_matcher(s1,s2):
    seq=difflib.SequenceMatcher(None, s1,s2)
    rt=round(seq.ratio(),7)
    l1=len(s1)
    l2=len(s2)
    if len(s1)==0 or len(s2)==0:
        rt=0
        rt_scaled=0
    else:
        rt_scaled=round(rt*max(l1,l2)/min(l1,l2),7)
    return rt, rt_scaled

In [207]:
def col_parser(clmn, automatic_spell_check_dict={}, remove_from_brackets=False,parse_material=False,add_space_stop_list=[]):
#     t0 = time()
    aa=list(set(list(clmn)))
    my_dict={}
    for i in range(0,len(aa)):
        my_dict[aa[i]]=str_parser(aa[i],automatic_spell_check_dict=automatic_spell_check_dict, remove_from_brackets=remove_from_brackets,\
                                    parse_material=parse_material,add_space_stop_list=add_space_stop_list)
        if (i % 10000)==0:
            print("parsed "+str(i)+" out of "+str(len(aa))+" unique values; "+str(round((time()-t0)/60,1))+" minutes")
    return clmn.map(lambda x: my_dict[x])

### create spell checker dictionary

In [208]:
add_space_stop_list=[]
uniq_brands=list(set(list(All_df['Brand'])))
for i in range(0,len(uniq_brands)):
    uniq_brands[i]=simple_parser(uniq_brands[i])
    if re.search(r'[a-z][A-Z][a-z]',uniq_brands[i])!=None:
        for word in uniq_brands[i].split():
            if re.search(r'[a-z][A-Z][a-z]',word)!=None:
                add_space_stop_list.append(word.lower())
add_space_stop_list=list(set(add_space_stop_list))      
print(len(add_space_stop_list)," words from brands in add_space_stop_list")

458  words from brands in add_space_stop_list


In [209]:
uniq_titles=list(set(list(All_df['product_title'])))
for i in range(0,len(uniq_titles)):
    uniq_titles[i]=simple_parser(uniq_titles[i])
    if re.search(r'[a-z][A-Z][a-z]',uniq_titles[i])!=None:
        for word in uniq_titles[i].split():
            if re.search(r'[a-z][A-Z][a-z]',word)!=None:
                add_space_stop_list.append(word.lower())    
add_space_stop_list=list(set(add_space_stop_list))      
print(len(add_space_stop_list) ," total words from brands and product titles in add_space_stop_list\n")

1722  total words from brands and product titles in add_space_stop_list



In [210]:
All_df['search_term'] = All_df['search_term'].map(lambda x: google_dict[x] if x in google_dict.keys() else x)

In [211]:
All_df['product_title_simpleparsed'] = All_df['product_title'].map(lambda x: simple_parser(x).lower())
All_df['search_term_simpleparsed'] = All_df['search_term'].map(lambda x: simple_parser(x).lower())

str_title = " ".join(list(All_df['product_title'].map(lambda x: simple_parser(x).lower())))
str_query = " ".join(list(All_df['search_term'].map(lambda x: simple_parser(x).lower())))

# create bigrams
bigrams_str_title = " ".join(list(All_df['product_title'].map(lambda x: create_bigrams(simple_parser(x).lower()))))
bigrams_set = set(bigrams_str_title.split())

In [212]:
my_dict={}
str1= str_title+" "+str_query
for word in list(set(list(str1.split()))):
    my_dict[word]={"title":0, "query":0, 'word':word}
for word in str_title.split():
    my_dict[word]["title"]+=1    
for word in str_query.split():
    my_dict[word]["query"]+=1

In [213]:
errors_dict={}
correct_dict={}
for word in my_dict.keys():
    if len(word)>=3 and len(re.sub('[^0-9]', '', word))==0:
        if my_dict[word]["title"]==0:
            if len(wn.synsets(word))>0 \
            or (word.endswith('s') and  (word[:-1] in my_dict.keys()) and my_dict[word[:-1]]["title"]>0)\
            or (word[-1]!='s' and (word+'s' in my_dict.keys()) and my_dict[word+'s']["title"]>0):
                1
            else:
                errors_dict[word]=my_dict[word]
        elif my_dict[word]["title"]>=5:
            correct_dict[word]=my_dict[word]

In [214]:
cnt=0
NN=len(errors_dict.keys())
# t0=time()
for i in range(0,len(errors_dict.keys())):
    word=sorted(errors_dict.keys())[i]
    cnt+=1
    lst=[]
    lst_tuple=[]
    suggested=False
    suggested_word=""
    rt_max=0
    
    # if only one word in query, use be more selective in choosing a correction
    min_query_len=min(All_df['search_term_simpleparsed'][All_df['search_term_simpleparsed'].map(lambda x: is_word_in_string(word,x))].map(lambda x: len(x.split())))
    delta=0.05*int(min_query_len<2)
    
    words_from_matched_titles=[item for item in \
        " ".join(list(set(All_df['product_title_simpleparsed'][All_df['search_term_simpleparsed'].map(lambda x: is_word_in_string(word,x))]))).split() \
        if len(item)>2 and len(re.sub('[^0-9]', '', item))==0]
    words_from_matched_titles=list(set(words_from_matched_titles))
    words_from_matched_titles.sort()
    
    source=""
    for bigram in bigrams_set:
        if bigram.replace("_","")==word:
            suggested=True
            suggested_word=bigram.replace("_"," ")
            source="from bigrams"
    if source=="":
        for correct_word in words_from_matched_titles: 
            rt, rt_scaled = seq_matcher(word,correct_word)
            #print correct_word, rt,rt_scaled
            
            if rt>0.75+delta or (len(word)<6 and rt>0.68+delta):
                lst.append(correct_word)
                lst_tuple.append((correct_word,my_dict[correct_word]["title"]))
                if rt>rt_max:
                    rt_max=rt
                    suggested=True
                    source="from matched products"
                    suggested_word=correct_word
                elif rt==rt_max and seq_matcher("".join(sorted(word)),"".join(sorted(correct_word)))[0]>seq_matcher("".join(sorted(word)),"".join(sorted(suggested_word)))[0]:
                    suggested_word=correct_word
                elif rt==rt_max:
                    suggested=False
                    source=""
        
    if source=="" and len(lst)==0:
        source="from all products"
        for correct_word in correct_dict.keys():
            rt, rt_scaled = seq_matcher(word,correct_word)
            #print correct_word, rt,rt_scaled
            if correct_dict[correct_word]["title"]>10 and (rt>0.8+delta or (len(word)<6 and rt>0.73+delta)):
                #print correct_word, rt,rt_scaled
                lst.append(correct_word)
                lst_tuple.append((correct_word,correct_dict[correct_word]["title"]))
                if rt>rt_max:
                    rt_max=rt
                    suggested=True
                    suggested_word=correct_word
                elif rt==rt_max and seq_matcher("".join(sorted(word)),"".join(sorted(correct_word)))[0]>seq_matcher("".join(sorted(word)),"".join(sorted(suggested_word)))[0]:
                    suggested_word=correct_word
                elif rt==rt_max: 
                    suggested=False
                    
    if suggested==True:
        errors_dict[word]["suggestion"]=suggested_word
        errors_dict[word]["others"]=lst_tuple
        errors_dict[word]["source"]=source
    else:
        errors_dict[word]["suggestion"]=""
        errors_dict[word]["others"]=lst_tuple
        errors_dict[word]["source"]=source

In [215]:
for word in my_dict.keys():
    if my_dict[word]['query']>0 and my_dict[word]['title']==0 \
    and len(re.sub('[^0-9]', '', word))!=0 and len(re.sub('[^a-z]', '', word))!=0:
        srch=re.search(r'(?<=^)[a-z][a-z][a-z]+(?=[0-9])',word)
        if srch!=None and len(wn.synsets(srch.group(0)))>0 \
        and len(re.sub('[^aeiou]', '', word))>0 and word[-1] in '0123456789': 
            errors_dict[word]=my_dict[word]
            errors_dict[word]["source"]="added space before digit"
            errors_dict[word]["suggestion"]=re.sub(r'(?<=^)'+srch.group(0)+r'(?=[a-zA-Z0-9])',srch.group(0)+' ',word)
            #print word, re.sub(r'(?<=^)'+srch.group(0)+r'(?=[a-zA-Z0-9])',srch.group(0)+' ',word)

In [216]:
##### load words for spell checker
spell_check_dict={}
for word in errors_dict.keys():
    if errors_dict[word]['suggestion']!="":
        spell_check_dict[word]=errors_dict[word]['suggestion']

### explore search_term and product title

In [219]:
t0 = time()
All_df['search_term_parsed']=col_parser(All_df['search_term'],automatic_spell_check_dict=spell_check_dict,\
            add_space_stop_list=[])#.map(lambda x: x.encode('utf-8'))
All_df['search_term_parsed_wospellcheck']=col_parser(All_df['search_term'],automatic_spell_check_dict={},\
            add_space_stop_list=[])#.map(lambda x: x.encode('utf-8'))

parsed 0 out of 24469 unique values; 0.0 minutes
parsed 10000 out of 24469 unique values; 7.8 minutes
parsed 20000 out of 24469 unique values; 15.7 minutes
parsed 0 out of 24469 unique values; 19.2 minutes
parsed 10000 out of 24469 unique values; 19.2 minutes
parsed 20000 out of 24469 unique values; 19.3 minutes


In [220]:
# def match_queries(q1,q2):
#     q1=re.sub('[^a-z\ ]', '', map(str,q1))
#     q2=re.sub('[^a-z\ ]', '', map(str,q2))
#     q1= " ".join([word[0:(len(word)-int(word[-1]=='s'))] for word in q1.split()])
#     q2= " ".join([word[0:(len(word)-int(word[-1]=='s'))] for word in q2.split()])
#     return difflib.SequenceMatcher(None, q1,q2).ratio()

In [221]:
# All_df['is_query_misspelled'] = All_df.apply(lambda x: \
#             match_queries(x['search_term_parsed'],x['search_term_parsed_wospellcheck']),axis=1)
# All_df = All_df.drop(['search_term_parsed_wospellcheck'],axis=1)    
# print('create dummy "is_query_misspelled" time:',round((time()-t0)/60,1) ,'minutes\n')

In [222]:
t0 = time()
All_df['product_title_parsed']=col_parser(All_df['product_title'],add_space_stop_list=[],\
                remove_from_brackets=True)#.map(lambda x: x.encode('utf-8'))
print('product_title parsing time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

parsed 0 out of 120453 unique values; 0.0 minutes
parsed 10000 out of 120453 unique values; 0.1 minutes
parsed 20000 out of 120453 unique values; 0.1 minutes
parsed 30000 out of 120453 unique values; 0.2 minutes
parsed 40000 out of 120453 unique values; 0.3 minutes
parsed 50000 out of 120453 unique values; 0.3 minutes
parsed 60000 out of 120453 unique values; 0.4 minutes
parsed 70000 out of 120453 unique values; 0.5 minutes
parsed 80000 out of 120453 unique values; 0.6 minutes
parsed 90000 out of 120453 unique values; 0.6 minutes
parsed 100000 out of 120453 unique values; 0.7 minutes
parsed 110000 out of 120453 unique values; 0.8 minutes
parsed 120000 out of 120453 unique values; 0.8 minutes
product_title parsing time: 0.8 minutes



### explore brand

In [223]:
### parse brand
All_df['brand_parsed'] = col_parser(All_df['Brand'].map(lambda x: re.sub('^[t|T]he ', '', x.replace(".N/A","").replace("N.A.","").replace("n/a","").replace("Generic Unbranded","").replace("Unbranded","").replace("Generic",""))),add_space_stop_list=add_space_stop_list)
list_brands = list(All_df['brand_parsed'])

All_df['brand_parsed'] = All_df['brand_parsed'].map(lambda x: replace_brand_dict[x] if x in replace_brand_dict.keys() else x)

parsed 0 out of 4285 unique values; 0.0 minutes


In [224]:
### count frequencies of brands in query and product_title
str_query=" : ".join(list(map(str,All_df['search_term_parsed']))).lower()
print("\nGenerating brand dict: How many times each brand appears in the dataset?")
brand_dict=get_attribute_dict(list_brands,str_query=str_query)


Generating brand dict: How many times each brand appears in the dataset?
500 out of 4266 unique attributes 1.1 minutes
1000 out of 4266 unique attributes 2.2 minutes
1500 out of 4266 unique attributes 3.4 minutes
2000 out of 4266 unique attributes 4.5 minutes
2500 out of 4266 unique attributes 5.6 minutes
3000 out of 4266 unique attributes 6.7 minutes
3500 out of 4266 unique attributes 7.9 minutes
4000 out of 4266 unique attributes 9.0 minutes


In [225]:
del_list=['aaa','off','impact','square','shelves','finish','ring','flood','dual','ball','cutter',\
'max','off','mat','allure','diamond','drive', 'edge','anchor','walls','universal','cat', 'dawn','ion','daylight',\
'roman', 'weed eater', 'restore', 'design', 'caddy', 'pole caddy', 'jet', 'classic', 'element', 'aqua',\
'terra', 'decora', 'ez', 'briggs', 'wedge', 'sunbrella',  'adorne', 'santa', 'bella', 'duck', 'hotpoint',\
'duck', 'tech', 'titan', 'powerwasher', 'cooper lighting', 'heritage', 'imperial', 'monster', 'peak', 
'bell', 'drive', 'trademark', 'toto', 'champion', 'shop vac', 'lava', 'jet', 'flood', \
'roman', 'duck', 'magic', 'allen', 'bunn', 'element', 'international', 'larson', 'tiki', 'titan', \
 'space saver', 'cutter', 'scotch', 'adorne', 'ball', 'sunbeam', 'fatmax', 'poulan', 'ring', 'sparkle', 'bissell', \
 'universal', 'paw', 'wedge', 'restore', 'daylight', 'edge', 'americana', 'wacker', 'cat', 'allure', 'bonnie plants', \
 'troy', 'impact', 'buffalo', 'adams', 'jasco', 'rapid dry', 'aaa', 'pole caddy', 'pac', 'seymour', 'mobil', \
 'mastercool', 'coca cola', 'timberline', 'classic', 'caddy', 'sentry', 'terrain', 'nautilus', 'precision', \
 'artisan', 'mural', 'game', 'royal', 'use', 'dawn', 'task', 'american line', 'sawtrax', 'solo', 'elements', \
 'summit', 'anchor', 'off', 'spruce', 'medina', 'shoulder dolly', 'brentwood', 'alex', 'wilkins', 'natural magic', \
 'kodiak', 'metro', 'shelter', 'centipede', 'imperial', 'cooper lighting', 'exide', 'bella', 'ez', 'decora', \
 'terra', 'design', 'diamond', 'mat', 'finish', 'tilex', 'rhino', 'crock pot', 'legend', 'leatherman', 'remove', \
 'architect series', 'greased lightning', 'castle', 'spirit', 'corian', 'peak', 'monster', 'heritage', 'powerwasher',\
 'reese', 'tech', 'santa', 'briggs', 'aqua', 'weed eater', 'ion', 'walls', 'max', 'dual', 'shelves', 'square',\
 'hickory', "vikrell", "e3", "pro series", "keeper", "coastal shower doors", 'cadet','church','gerber','glidden',\
 'cooper wiring devices', 'border blocks', 'commercial electric', 'pri','exteria','extreme', 'veranda',\
 'gorilla glue','gorilla','shark','wen']
del_list=list(set(list(del_list)))

for key in del_list:
    if key in brand_dict.keys():
        del(brand_dict[key])

In [226]:
# brand_dict

In [227]:
# for item in brand_dict.keys():
#     if (brand_dict[item]['cnt_attribute']>=3 and brand_dict[item]['cnt_query']>=1) \
#     or (brand_dict[item]['cnt_attribute'])>=8:
#         1
#     else:
#         del(brand_dict[item])

In [228]:
brand_df=pd.DataFrame(brand_dict).transpose().sort(['cnt_query'], ascending=[1])

### explore Material

In [232]:
def change_material(s):
    replace_dict={'Medium Density Fiberboard (MDF)':'mdf', 'High Density Fiberboard (HDF)':'hdf',\
    'Fibre Reinforced Polymer (FRP)': 'frp', 'Acrylonitrile Butadiene Styrene (ABS)': 'abs',\
    'Cross-Linked Polyethylene (PEX)':'pex', 'Chlorinated Poly Vinyl Chloride (CPVC)': 'cpvc',\
    'PVC (vinyl)': 'pvc','Thermoplastic rubber (TPR)':'tpr','Poly Lactic Acid (PLA)': 'pla',\
    '100% Polyester':'polyester','100% UV Olefin':'olefin', '100% BCF Polypropylene': 'polypropylene',\
    '100% PVC':'pvc'}
        
    if s in replace_dict.keys():
        s=replace_dict[s]
    return s
All_df['Material'] = All_df['Material'].map(lambda x: change_material(x))

In [233]:
# do_copy = True
# if do_copy:
dict_materials = {}
key_list = material_df['product_uid'].keys()
for i in range(0,len(key_list)):
    if material_df['product_uid'][key_list[i]] not in dict_materials.keys():
        dict_materials[material_df['product_uid'][key_list[i]]]={}
        dict_materials[material_df['product_uid'][key_list[i]]]['product_uid']=material_df['product_uid'][key_list[i]]
        dict_materials[material_df['product_uid'][key_list[i]]]['cnt']=1
        dict_materials[material_df['product_uid'][key_list[i]]]['material']=material_df['Material'][key_list[i]]
    else:
        ##print key_list[i]
        dict_materials[material_df['product_uid'][key_list[i]]]['material']=dict_materials[material_df['product_uid'][key_list[i]]]['material']+' '+material_df['Material'][key_list[i]]
        dict_materials[material_df['product_uid'][key_list[i]]]['cnt']+=1
    if (i % 10000)==0:
        print (i)
                       
df_materials=pd.DataFrame(dict_materials).transpose()
else:
    All_df['Material'] = All_df['Material'].map(lambda x: change_material(x))
    material_df['value'] = material_df['Material'].map(lambda x: change_material(x))
    key_list=material_df['product_uid'].keys()
    dict_materials = {}
    key_list=material_df['product_uid'].keys()
    for i in range(0,len(key_list)):
        if material_df['product_uid'][key_list[i]] not in dict_materials.keys():
            dict_materials[material_df['product_uid'][key_list[i]]]={}
            dict_materials[material_df['product_uid'][key_list[i]]]['product_uid']=material_df['product_uid'][key_list[i]]
            dict_materials[material_df['product_uid'][key_list[i]]]['cnt']=1
            dict_materials[material_df['product_uid'][key_list[i]]]['material']=material_df['value'][key_list[i]]
        else:
            ##print key_list[i]
            dict_materials[material_df['product_uid'][key_list[i]]]['material']=dict_materials[material_df['product_uid'][key_list[i]]]['material']+' '+material_df['value'][key_list[i]]
            dict_materials[material_df['product_uid'][key_list[i]]]['cnt']+=1
        if (i % 10000)==0:
            print(i)



0
10000
20000
30000
40000


In [234]:
All_df = All_df.drop('Material',axis=1)
All_df = pd.merge(All_df, material_df, how='left', on='product_uid')
All_df['Material']=All_df['Material'].fillna("")#.map(lambda x: x.encode('utf-8'))
All_df['material_parsed']=col_parser(All_df['Material'].map(lambda x: x.replace("Other","").replace("*","")), parse_material=True,add_space_stop_list=[])


parsed 0 out of 1851 unique values; 9.9 minutes


In [235]:
### list of all materials
list_materials=list(All_df['material_parsed'].map(lambda x: x.lower())) 
### count frequencies of materials in query and product_title
print("\nGenerating material dict: How many times each material appears in the dataset?")
material_dict=get_attribute_dict(list_materials,str_query=str_query)
### create dataframe and save to file
material_df_1=pd.DataFrame(material_dict).transpose()


Generating material dict: How many times each material appears in the dataset?
500 out of 1668 unique attributes 1.1 minutes
1000 out of 1668 unique attributes 2.3 minutes
1500 out of 1668 unique attributes 3.4 minutes


### generate whether material and brand in query and title

In [236]:
def getremove_brand_or_material_from_str(s,df, replace_brand_dict={}):
    items_found=[]
    df=df.sort_values(['nwords'],ascending=[0])
    key_list=df['nwords'].keys()
    #start with several-word brands or materials
    #assert df['nwords'][key_list[0]]>1
    for i in range(0,len(key_list)):
        item=df['name'][key_list[i]]
        if item in s:
            if re.search(r'\b'+item+r'\b',s)!=None:
                s=re.sub(r'\b'+item+r'\b', '', s)
                if item in replace_brand_dict.keys():
                    items_found.append(replace_brand_dict[item])
                else:
                    items_found.append(item)

    return " ".join(s.split()), ";".join(items_found)

In [ ]:
aa=list(set(list(All_df['search_term_parsed'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=getremove_brand_or_material_from_str(aa[i],brand_df)
    if (i % 5000)==0:
        print("Extracted brands from",i,"out of",len(aa),"unique search terms; ", str(round((time()-t0)/60,1)),"minutes")
All_df['search_term_tuple']= All_df['search_term_parsed'].map(lambda x: my_dict[x])
All_df['search_term_parsed_woBrand']= All_df['search_term_tuple'].map(lambda x: x[0])
All_df['brands_in_search_term']= All_df['search_term_tuple'].map(lambda x: x[1])
print('extract brands from query time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

All_df['search_term_tuple']= All_df['search_term_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df_1))
All_df['search_term_parsed_woBM']= All_df['search_term_tuple'].map(lambda x: x[0])
All_df['materials_in_search_term']= All_df['search_term_tuple'].map(lambda x: x[1])
All_df = All_df.drop('search_term_tuple',axis=1)
print('extract materials from query time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

In [254]:
All_df['search_term_parsed_woBrand']= All_df['search_term_parsed_woBrand'].fillna('')

In [255]:
All_df['search_term_tuple']= All_df['search_term_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df_1))
All_df['search_term_parsed_woBM']= All_df['search_term_tuple'].map(lambda x: x[0])
All_df['materials_in_search_term']= All_df['search_term_tuple'].map(lambda x: x[1])
All_df = All_df.drop('search_term_tuple',axis=1)
print('extract materials from query time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

extract materials from query time: 351.5 minutes



In [256]:
aa=list(set(list(All_df['product_title_parsed'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=getremove_brand_or_material_from_str(aa[i],brand_df)
    if (i % 5000)==0:
        print("Extracted brands from",i,"out of",len(aa),"unique product titles; ", str(round((time()-t0)/60,1)),"minutes")

All_df['product_title_tuple']= All_df['product_title_parsed'].map(lambda x: my_dict[x])
All_df['product_title_parsed_woBrand']= All_df['product_title_tuple'].map(lambda x: x[0])
All_df['brands_in_product_title']= All_df['product_title_tuple'].map(lambda x: x[1])
print('extract brands from product title time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()


All_df['product_title_tuple']= All_df['product_title_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df_1))
All_df['product_title_parsed_woBM']= All_df['product_title_tuple'].map(lambda x: x[0])
All_df['materials_in_product_title']= All_df['product_title_tuple'].map(lambda x: x[1])
All_df = All_df.drop('product_title_tuple',axis=1)
print('extract materials from product titles time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()


Extracted brands from 0 out of 118401 unique product titles;  0.0 minutes
Extracted brands from 5000 out of 118401 unique product titles;  7.3 minutes
Extracted brands from 10000 out of 118401 unique product titles;  14.6 minutes
Extracted brands from 15000 out of 118401 unique product titles;  21.9 minutes
Extracted brands from 20000 out of 118401 unique product titles;  29.1 minutes
Extracted brands from 25000 out of 118401 unique product titles;  36.4 minutes
Extracted brands from 30000 out of 118401 unique product titles;  43.6 minutes
Extracted brands from 35000 out of 118401 unique product titles;  50.9 minutes
Extracted brands from 40000 out of 118401 unique product titles;  58.1 minutes
Extracted brands from 45000 out of 118401 unique product titles;  65.4 minutes
Extracted brands from 50000 out of 118401 unique product titles;  72.6 minutes
Extracted brands from 55000 out of 118401 unique product titles;  79.9 minutes
Extracted brands from 60000 out of 118401 unique product ti

In [257]:
### tagging
All_df['search_term_tokens'] =col_tagger(All_df['search_term_parsed_woBM'])
print('search term tagging time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()
All_df['product_title_tokens'] =col_tagger(All_df['product_title_parsed_woBM'])
print('product title tagging time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

tagged 20000 out of 240761 total rows; 0.1 minutes
tagged 40000 out of 240761 total rows; 0.2 minutes
tagged 60000 out of 240761 total rows; 0.3 minutes
tagged 80000 out of 240761 total rows; 0.4 minutes
tagged 100000 out of 240761 total rows; 0.6 minutes
tagged 120000 out of 240761 total rows; 0.7 minutes
tagged 140000 out of 240761 total rows; 0.8 minutes
tagged 160000 out of 240761 total rows; 0.9 minutes
tagged 180000 out of 240761 total rows; 1.0 minutes
tagged 200000 out of 240761 total rows; 1.1 minutes
tagged 220000 out of 240761 total rows; 1.2 minutes
tagged 240000 out of 240761 total rows; 1.3 minutes
tagged 240761 out of 240761 total rows; 1.3 minutes
search term tagging time: 1.3 minutes

tagged 20000 out of 240761 total rows; 0.3 minutes
tagged 40000 out of 240761 total rows; 0.6 minutes
tagged 60000 out of 240761 total rows; 0.9 minutes
tagged 80000 out of 240761 total rows; 1.2 minutes
tagged 100000 out of 240761 total rows; 1.5 minutes
tagged 120000 out of 240761 total

### explore Bullets

In [259]:
All_df['Bullet'] = All_df['Bullet'].fillna('')

In [ ]:
### Parsing
All_df['attribute_bullets_parsed'] = All_df['Bullet'].map(lambda x:str_parser(x,add_space_stop_list=[]))
print('attribute bullets parsing time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

### Extracting brands...
All_df['attribute_bullets_tuple']= All_df['attribute_bullets_parsed'].map(lambda x: getremove_brand_or_material_from_str(x,brand_df))
All_df['attribute_bullets_parsed_woBrand']= All_df['attribute_bullets_tuple'].map(lambda x: x[0])
All_df['brands_in_attribute_bullets']= All_df['attribute_bullets_tuple'].map(lambda x: x[1])
print('extract brands from attribute_bullets time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

## ... and materials from text...
All_df['attribute_bullets_tuple']= All_df['attribute_bullets_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df_1))
All_df['attribute_bullets_parsed_woBM']= All_df['attribute_bullets_tuple'].map(lambda x: x[0])
All_df['materials_in_attribute_bullets']= All_df['attribute_bullets_tuple'].map(lambda x: x[1])
All_df = All_df.drop(['attribute_bullets_tuple'],axis=1)
print('extract materials from attribute_bullets time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

### ... and tagging text using NLTK
All_df['attribute_bullets_tokens'] =col_tagger(All_df['attribute_bullets_parsed_woBM'])
print('attribute bullets tagging time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

In [273]:
All_df['attribute_bullets_parsed'] = All_df['attribute_bullets_parsed'].fillna('')

### explore Description

In [336]:
### Parsing
pro_descr['product_description_parsed'] = pro_descr['product_description'].map(lambda x:str_parser(x,add_space_stop_list=add_space_stop_list))#.encode('utf-8'))
print('product description parsing time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

product description parsing time: 100.6 minutes



In [337]:
###
pro_descr['product_description_tuple']= pro_descr['product_description_parsed'].map(lambda x: getremove_brand_or_material_from_str(x,brand_df))
pro_descr['product_description_parsed_woBrand']= pro_descr['product_description_tuple'].map(lambda x: x[0])
pro_descr['brands_in_product_description']= pro_descr['product_description_tuple'].map(lambda x: x[1])
print('extract brands from product_description time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

extract brands from product_description time: 189.8 minutes



In [338]:
### ... and materials from text...
pro_descr['product_description_tuple']= pro_descr['product_description_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df_1))
pro_descr['product_description_parsed_woBM']= pro_descr['product_description_tuple'].map(lambda x: x[0])
pro_descr['materials_in_product_description']= pro_descr['product_description_tuple'].map(lambda x: x[1])
pro_descr=pro_descr.drop(['product_description_tuple'],axis=1)
print('extract materials from product_description time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

extract materials from product_description time: 77.5 minutes



In [339]:
### ... and tagging text using NLTK
pro_descr['product_description_tokens'] = col_tagger(pro_descr['product_description_parsed_woBM'])
print('product decription tagging time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

# pro_descr['product_description']= pro_descr['product_description']#.map(lambda x: x.encode('utf-8'))

tagged 20000 out of 124428 total rows; 1.8 minutes
tagged 40000 out of 124428 total rows; 3.6 minutes
tagged 60000 out of 124428 total rows; 5.4 minutes
tagged 80000 out of 124428 total rows; 7.2 minutes
tagged 100000 out of 124428 total rows; 9.0 minutes
tagged 120000 out of 124428 total rows; 10.7 minutes
tagged 124428 out of 124428 total rows; 11.1 minutes
product decription tagging time: 11.1 minutes



### Get importand words from query and title

In [287]:
def cut_product_title(s):
    s=s.lower()
    s = re.sub('&amp;', '&', s)
    s = re.sub('&nbsp;', '', s)
    s = re.sub('&#39;', '', s)
    s = re.sub(r'(?<=[0-9]),[\ ]*(?=[0-9])', '', s)
    s = re.sub(r'(?<=\))(?=[a-zA-Z0-9])', ' ', s) # add space between parentheses and letters
    s = re.sub(r'(?<=[a-zA-Z0-9])(?=\()', ' ', s) # add space between parentheses and letters
    s = s.replace(";",". ")
    s = s.replace(":"," . ")
    s=s.replace("&"," and ") 
    s = re.sub('[^a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,\+]', ' ', s)
    s= " ".join(s.split())
    s = re.sub(r'(?<=[0-9])\.\ ', ' ', s)
    s = re.sub(r'(?<=\ in)\.(?=[a-zA-Z])', '. ', s)
    
    s=replace_in_parser(s)
    
    s = re.sub(r'\-discontinued', '', s)
    s = re.sub(r' \+ free app(?=$)', '', s)
    s = s.replace("+"," ")    
    s = re.sub('\([a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]+?\)', '', s)
    #s= re.sub('[\(\)]', '', s)
    if " - " in s:
        #srch=re.search(r'(?<= - )[a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]+',s) 
        if re.search(r'(\d|\.|mm|cm|in|ft|mhz|volt|\bhp|\bl|oz|lb|gal) \- \d',s)==None \
            and re.search(r' (sign|carpet|decal[s]*|figure[s]*)(?=$)',s)==None and re.search(r'\d \- (way\b|day\b)',s)==None: 
        #if ' - ' is found and the string doesnt end with word 'sign' or 'carpet' or 'decal' and not string '[0-9] - way' found
            s = re.sub(r' - [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s) #greedy regular expression
    if "uilt in" not in s and "uilt In" not in s:
        s = re.sub(r'(?<=[a-zA-Z\%\$\#\@\&\/\.\*])[\ ]+[I|i]n [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
    s = s.replace(" - "," ")
    if re.search(r' (sign|decal[s]*|figure[s]*)(?=$)',s)==None:    
        s = re.sub(r'(?<=[a-zA-Z0-9\%\$\#\@\&\/\.\*])[\ ]+[W|w]ith [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
        s = re.sub(r'(?<=[a-zA-Z0-9\%\$\#\@\&\/\.\*])[\ ]+[W|w]ithout [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
    s = re.sub(r'(?<=[a-zA-Z\%\$\#\@\&\/\.\*])[\ ]+[w]/[\ a-z0-9][a-z0-9][a-z0-9\.][a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
    
    if " fits for " not in s and " for fits " not in s:    
        s = re.sub(r'(?<=[a-zA-Z0-9\%\$\#\@\&\/\.\*])[\ ]+fits [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)    
    if " for lease " not in s and re.search(r' (sign|decal[s]*|figure[s]*)(?=$)',s)==None:
        s = re.sub(r'(?<=[a-zA-Z0-9\%\$\#\@\&\/\.\*])[\ ]+[F|f]or [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
    s = re.sub(r'(?<=[a-zA-Z0-9\%\$\#\@\&\/\.\*])[\ ]+[T|t]hat [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)


    s = re.sub(r' on (wheels|a pallet|spool|bracket|3 in|blue post|360|track|spike|rock|lamp|11 in|2 in|pedestal|square base|tub|steel work)[a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)]*', '', s)
    s = re.sub(r' on (plinth|insulator|casters|pier base|reel|fireplace|moon|bracket|24p ebk|zinc spike|mailbox|cream chand|blue post)[a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)]*', '', s)
 
    s = re.sub(r'(?<= on white stiker)[a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
    s = re.sub(r' on [installing][a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
    if "," in s:
        srch=re.search(r'(?<=, )[a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*',s)  #greedy regular expression
        if srch!=None:
            if len(re.sub('[^a-zA-Z\ ]', '', srch.group(0)))<25:
                s = re.sub(r', [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
    s = re.sub(r'(?<=recessed door reinforcer), [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]*', '', s)
    #s = re.sub(r'(?<=[a-zA-Z0-9]),\ [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)]*', '', s)
    s = re.sub(r'(?<=[a-zA-Z\%\$\#\@\&\/\.\*]) [F|f]eaturing [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)]*', '', s)
    s = re.sub(r'(?<=[a-zA-Z\%\$\#\@\&\/\.\*]) [I|i]ncludes [a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)]*', '', s)
    s = re.sub(' [\#]\d+[\-\d]*[\,]*', '', s)  
    s = re.sub(r'(?<=[a-zA-Z\ ])\/(?=[a-zA-Z])', ' ', s)
    s = re.sub(r'(?<=[a-zA-Z\ ])\-(?=[a-zA-Z])', ' ', s)
    s = s.replace(",",". ")
    s = s.replace("..",".")
    s = s.replace("..",".")
    s = s.replace("*","")
    return " ".join([word.replace("-","") for word in s.split() if re.search(r'\d\-\d',word)==None])

In [288]:
not_keyword_list=list(brand_df['name'][brand_df['nwords']==1])
for item in add_space_stop_list:
    if len(wn.synsets(item,pos=wn.NOUN))==0:
        not_keyword_list.append(item)

In [289]:
def get_key_words(tag_list,wordtag_list, string_output=False,out_list=not_keyword_list[:]):
    i=len(tag_list)
    in_list=['tv','downrod', 'sillcock', 'shelving', 'luminaire', 'paracord', 'ducting', \
    'recyclamat', 'rebar', 'spackling', 'hoodie', 'placemat', 'innoculant', 'protectant', \
    'colorant', 'penetrant', 'attractant', 'bibb', 'nosing', 'subflooring', 'torchiere', 'thhn',\
    'lantern','epoxy','cloth','trim','adhesive','light','lights','saw','pad','polish','nose','stove',\
    'elbow','elbows','lamp','door','doors','pipe','bulb','wood','woods','wire','sink','hose','tile','bath','table','duct',\
    'windows','mesh','rug','rugs','shower','showers','wheels','fan','lock','rod','mirror','cabinet','shelves','paint',\
    'plier','pliers','set','screw','lever','bathtub','vacuum','nut', 'nipple','straw','saddle','pouch','underlayment',\
    'shade','top', 'bulb', 'bulbs', 'paint', 'oven', 'ranges', 'sharpie', 'shed', 'faucet',\
    'finish','microwave', 'can', 'nozzle', 'grabber', 'tub', 'angles','showerhead', 'dehumidifier', 'shelving', 'urinal', 'mdf']

    out_list= out_list +['free','height', 'width', 'depth', 'model','pcs', 'thick','pack','adhesive','steel','cordless', 'aaa' 'b', 'nm', 'hc', 'insulated','gll', 'nutmeg',\
    'pnl', 'sotc','withe','stainless','chrome','beige','max','acrylic', 'cognac', 'cherry', 'ivory','electric','fluorescent', 'recessed', 'matte',\
    'propane','sku','brushless','quartz','gfci','shut','sds','value','brown','white','black','red','green','yellow','blue','silver','pink',\
    'gray','gold','thw','medium','type','flush',"metaliks", 'metallic', 'amp','btu','gpf','pvc','mil','gcfi','plastic', 'vinyl','aaa',\
    'aluminum','brass','antique', 'brass','copper','nickel','satin','rubber','porcelain','hickory','marble','polyacrylic','golden','fiberglass',\
    'nylon','lmapy','maple','polyurethane','mahogany','enamel', 'enameled', 'linen','redwood', 'sku','oak','quart','abs','travertine', 'resin',\
    'birch','birchwood','zinc','pointe','polycarbonate', 'ash', 'wool', 'rockwool', 'teak','alder','frp','cellulose','abz', 'male', 'female', 'used',\
    'hepa','acc','keyless','aqg','arabesque','polyurethane', 'polyurethanes','ardex','armorguard','asb', 'motion','adorne','fatpack',\
    'fatmax','feet','ffgf','fgryblkg', 'douglas', 'fir', 'fleece','abba', 'nutri', 'thermal','thermoclear', 'heat', 'water', 'systemic',\
    'heatgasget', 'cool', 'fusion', 'awg', 'par', 'parabolic', 'tpi', 'pint', 'draining', 'rain', 'cost', 'costs', 'costa','ecostorage',
    'mtd', 'pass', 'emt', 'jeld', 'npt', 'sch', 'pvc', 'dusk', 'dawn', 'lathe','lows','pressure', 'round', 'series','impact', 'resistant','outdoor',\
    'off', 'sawall', 'elephant', 'ear', 'abb', 'baby', 'feedback', 'fastback','jumbo', 'flexlock', 'instant', 'natol', 'naples','florcant',\
    'canna','hammock', 'jrc', 'honeysuckle', 'honey', 'serrano','sequoia', 'amass', 'ashford', 'gal','gas', 'gasoline', 'compane','occupancy',\
    'home','bakeware', 'lite', 'lithium', 'golith','gxwh',  'wht', 'heirloom', 'marine', 'marietta', 'cambria', 'campane','birmingham',\
    'bellingham','chamois', 'chamomile', 'chaosaw', 'chanpayne', 'thats', 'urethane', 'champion', 'chann', 'mocha', 'bay', 'rough',\
    'undermount', 'price', 'prices', 'way', 'air', 'bazaar', 'broadway', 'driveway', 'sprayway', 'subway', 'flood', 'slate', 'wet',\
    'clean', 'tweed', 'weed', 'cub', 'barb', 'salem', 'sale', 'sales', 'slip', 'slim', 'gang', 'office', 'allure', 'bronze', 'banbury',\
    'tuscan','tuscany', 'refinishing', 'fleam','schedule', 'doeskin','destiny', 'mean', 'hide', 'bobbex', 'pdi', 'dpdt', 'tri', 'order',\
    'kamado','seahawks','weymouth', 'summit','tel','riddex', 'alick','alvin', 'ano', 'assy', 'grade', 'barranco', 'batte','banbury',\
    'mcmaster', 'carr', 'ccl', 'china', 'choc', 'colle', 'cothom', 'cucbi', 'cuv', 'cwg', 'cylander', 'cylinoid', 'dcf', 'number', 'ultra',\
    'diat','discon', 'disconnect', 'plantation', 'dpt', 'duomo', 'dupioni', 'eglimgton', 'egnighter','ert','euroloft', 'everready',\
    'felxfx', 'financing', 'fitt', 'fosle', 'footage', 'gpf','fro', 'genis', 'giga', 'glu', 'gpxtpnrf', 'size', 'hacr', 'hardw',\
    'hexagon', 'hire', 'hoo','number','cosm', 'kelston', 'kind', 'all', 'semi', 'gloss', 'lmi', 'luana', 'gdak', 'natol', 'oatu',\
    'oval', 'olinol', 'pdi','penticlea', 'portalino', 'racc', 'rads', 'renat', 'roc', 'lon', 'sendero', 'adora', 'sleave', 'swu',
    'tilde', 'cordoba', 'tuvpl','yel', 'acacia','mig','parties','alkaline','plexiglass', 'iii', 'watt']
    
    output_list=[]
    if i>0:
        finish=False
        started = False
        while not finish:
            i-=1
            
            if started==False:
                if (wordtag_list[i][0] not in out_list) \
                and (wordtag_list[i][0] in in_list \
                    or (re.search(r'(?=[e|o]r[s]*\b)',wordtag_list[i][0])!=None and re.search(r'\d+',wordtag_list[i][0])==None) \
                    or (len(wordtag_list[i][0])>2 and re.search(r'\d+',wordtag_list[i][0])==None and len(wn.synsets(wordtag_list[i][0],pos=wn.NOUN))>0 \
                        and (wordtag_list[i][1] in ['NN', 'NNS','VBG'] or tag_list[i][1] in ['NN', 'NNS','VBG']) \
                            and len(re.sub('[^aeiouy]', '', wordtag_list[i][0]))>0 )): #exclude VBD
                    started = True
                    output_list.insert(0,wordtag_list[i])
                        # handle exceptions below
                        # 'iron' only with -ing is OK: soldering iron, seaming iron
                    if i>1 and wordtag_list[i][0] in ['iron','irons'] and re.search(r'ing\b',wordtag_list[i-1][0])==None:
                        output_list=[]
                        started = False    
            else:

                if tag_list[i][1] in ['NN','NNP', 'NNPS', 'NNS']:
                    if len(re.sub('[^0-9]', '', tag_list[i][0]))==0 and \
                    len(re.sub('[^a-zA-Z0-9\-]', '', tag_list[i][0]))>2 \
                    and tag_list[i][0] not in ['amp','btu','gpf','pvc','mil','watt','gcfi']\
                    and (len(wn.synsets(tag_list[i][0]))>0 or re.search(r'(?=[e|o]r[s]*\b)',tag_list[i][0])!=None):
                        output_list.insert(0,tag_list[i])
                elif tag_list[i][0]=='and':
                    output_list.insert(0,tag_list[i])
                    started=False
                else:
                    if tag_list[max(0,i-1)][0]!="and" and (tag_list[i][1] not in ['VBD', 'VBN']):
                        finish=True
                    if tag_list[i][1] in ['JJ','JJS', 'JJR', 'RB', 'RBS', 'RBR', 'VBG', 'VBD', 'VBN','VBP']:
                        if len(re.sub('[^0-9]', '', tag_list[i][0]))==0 and \
                        len(re.sub('[^a-zA-Z0-9\-]', '', tag_list[i][0]))>2 \
                        and tag_list[i][0] not in ['amp','btu','gpf','pvc','mil','watt','gcfi']\
                        and len(wn.synsets(tag_list[i][0]))>0:
                            output_list.insert(0,tag_list[i])
                
            if i==0:
                finish=True
    if string_output==True:
        return " ".join([tag[0] for tag in output_list])
    else:
        return output_list

In [290]:
All_df['product_title_cut'] = All_df['product_title'].map(lambda x: cut_product_title(x))#.encode('utf-8')

In [291]:
### ... and that is why we have to remove the brand names again
aa=list(set(list(All_df['product_title_cut'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=getremove_brand_or_material_from_str(aa[i],brand_df)
    if (i % 5000)==0:
        print("processed "+str(i)+" out of "+str(len(aa))+" unique cut product titles; "+str(round((time()-t0)/60,1))+" minutes")
All_df['product_title_cut_tuple'] = All_df['product_title_cut'].map(lambda x: my_dict[x])
All_df['product_title_cut_woBrand'] = All_df['product_title_cut_tuple'].map(lambda x: x[0])
All_df = All_df.drop(['product_title_cut_tuple'],axis=1)
print('extract brands from cut product title:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

processed 0 out of 108307 unique cut product titles; 1.2 minutes
processed 5000 out of 108307 unique cut product titles; 8.6 minutes
processed 10000 out of 108307 unique cut product titles; 16.0 minutes
processed 15000 out of 108307 unique cut product titles; 23.5 minutes
processed 20000 out of 108307 unique cut product titles; 30.9 minutes
processed 25000 out of 108307 unique cut product titles; 38.3 minutes
processed 30000 out of 108307 unique cut product titles; 45.7 minutes
processed 35000 out of 108307 unique cut product titles; 53.2 minutes
processed 40000 out of 108307 unique cut product titles; 60.7 minutes
processed 45000 out of 108307 unique cut product titles; 68.2 minutes
processed 50000 out of 108307 unique cut product titles; 75.7 minutes
processed 55000 out of 108307 unique cut product titles; 83.2 minutes
processed 60000 out of 108307 unique cut product titles; 90.7 minutes
processed 65000 out of 108307 unique cut product titles; 98.1 minutes
processed 70000 out of 1083

In [293]:
### Tagging two times: full sentences and separate words
All_df['product_title_cut_tokens'] =col_tagger(All_df['product_title_cut_woBrand'])
All_df['product_title_cut_wordtokens'] =col_wordtagger(All_df['product_title_cut_woBrand'])

### the same steps for search term, but we now we continue with the preprocessed resuts
### since punctuation is not as important in query as it is in product title
All_df['search_term_cut_woBrand']= All_df['search_term_parsed_woBrand'].map(lambda x: cut_product_title(x))#.encode('utf-8'))
All_df['search_term_cut_tokens'] =col_tagger(All_df['search_term_cut_woBrand'])
All_df['search_term_cut_wordtokens'] =col_wordtagger(All_df['search_term_cut_woBrand'])

### Transform tags into text, it may look like unecessary step.
### But in our work we have to frequently save processing results and recover tags from text.
### Here this transformation is used to make the _tokens variables compatibe with 
### parser_mystr2tuple() function 
All_df['search_term_cut_tokens']=All_df['search_term_cut_tokens'].map(lambda x: str(x))
All_df['search_term_cut_wordtokens']=All_df['search_term_cut_wordtokens'].map(lambda x: str(x))
All_df['product_title_cut_tokens']=All_df['product_title_cut_tokens'].map(lambda x: str(x))
All_df['product_title_cut_wordtokens']=All_df['product_title_cut_wordtokens'].map(lambda x: str(x))


All_df['search_term_keys']=All_df.apply(lambda x: \
            get_key_words(parser_mystr2tuple(x['search_term_cut_tokens']),parser_mystr2tuple(x['search_term_cut_wordtokens']),string_output=True),axis=1)
All_df['product_title_keys']=All_df.apply(lambda x: \
            get_key_words(parser_mystr2tuple(x['product_title_cut_tokens']),parser_mystr2tuple(x['product_title_cut_wordtokens']),string_output=True),axis=1)




tagged 20000 out of 240761 total rows; 0.4 minutes
tagged 40000 out of 240761 total rows; 0.7 minutes
tagged 60000 out of 240761 total rows; 1.1 minutes
tagged 80000 out of 240761 total rows; 1.4 minutes
tagged 100000 out of 240761 total rows; 1.7 minutes
tagged 120000 out of 240761 total rows; 2.1 minutes
tagged 140000 out of 240761 total rows; 2.4 minutes
tagged 160000 out of 240761 total rows; 2.8 minutes
tagged 180000 out of 240761 total rows; 3.1 minutes
tagged 200000 out of 240761 total rows; 3.5 minutes
tagged 220000 out of 240761 total rows; 3.8 minutes
tagged 240000 out of 240761 total rows; 4.2 minutes
tagged 240761 out of 240761 total rows; 4.2 minutes
wordtagged 20000 out of 240761 total rows; 0.4 minutes
wordtagged 40000 out of 240761 total rows; 0.7 minutes
wordtagged 60000 out of 240761 total rows; 1.1 minutes
wordtagged 80000 out of 240761 total rows; 1.5 minutes
wordtagged 100000 out of 240761 total rows; 1.8 minutes
wordtagged 120000 out of 240761 total rows; 2.2 minu

In [294]:
def get_last_words_from_parsed_title(s):
    words=s.split()
    if len(words)==0:
        last_word=""
        word_before_last=""
        word2_before_last=""
    else:
        last_word=words[len(words)-1]
        word_before_last=""
        word2_before_last=""
        if len(words)>1:
            word_before_last=words[len(words)-2]
            if word_before_last=="and":
                word_before_last=""
            if len(words)>2 and word_before_last!="and":
                word2_before_last=words[len(words)-3]
                if word2_before_last=="and":
                    word2_before_last=""
    return last_word, word_before_last, word2_before_last

def get_last_words_from_parsed_query(s,last_word_in_title):
    words=s.split()
    if len(words)==0:
        last_word=""
        word_before_last=""
        word2_before_last=""
    else:
        last_word=words[len(words)-1]
        word_before_last=""
        word2_before_last=""
        if len(words)>1:
            word_before_last=words[len(words)-2]
            
            if len(words)>2 and word_before_last!="and":
                word2_before_last=words[len(words)-3]
                if word2_before_last=="and":
                    word2_before_last=""
                    
            if word_before_last=="and":
                word_before_last=""
                if len(words)>2:
                    cmp_word=words[len(words)-3]
                    sm1=find_similarity(last_word,last_word_in_title)[0]
                    sm2=find_similarity(cmp_word,last_word_in_title)[0]
                    if sm1<sm2:
                        last_word=cmp_word
                        if len(words)>3:
                            word_before_last=words[len(words)-4]
               
            
    return last_word, word_before_last, word2_before_last

In [295]:
### get trigram from product title
All_df['product_title_thekey_tuple']=All_df['product_title_keys'].map(lambda x: get_last_words_from_parsed_title(x))
All_df['product_title_thekey']=All_df['product_title_thekey_tuple'].map(lambda x: x[0])
All_df['product_title_beforethekey']=All_df['product_title_thekey_tuple'].map(lambda x: x[1])
All_df['product_title_before2thekey']=All_df['product_title_thekey_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['product_title_thekey_tuple'],axis=1)


### get trigram from query
All_df['search_term_thekey_tuple']=All_df.apply(lambda x: \
            get_last_words_from_parsed_query(x['search_term_keys'],x['product_title_thekey']),axis=1)
#All_df['thekey_info']=All_df['search_term_keys']+"\t"+All_df['product_title_thekey']
#All_df['search_term_thekey_tuple']=All_df['thekey_info'].map(lambda x: get_last_words_from_parsed_query(x.split("\t")[0],x.split("\t")[1]))
All_df['search_term_thekey']=All_df['search_term_thekey_tuple'].map(lambda x: x[0])
All_df['search_term_beforethekey']=All_df['search_term_thekey_tuple'].map(lambda x: x[1])
All_df['search_term_before2thekey']=All_df['search_term_thekey_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['search_term_thekey_tuple'],axis=1)

print('extracting important words time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

extracting important words time: 53.7 minutes



In [299]:
All_df['attribute_bullets_stemmed']=All_df['attribute_bullets_parsed'].map(lambda x:str_stemmer_wo_parser(x))
All_df['attribute_bullets_stemmed_woBM']=All_df['attribute_bullets_parsed_woBM'].map(lambda x:str_stemmer_wo_parser(x))
All_df['attribute_bullets_stemmed_woBrand']=All_df['attribute_bullets_parsed_woBrand'].map(lambda x:str_stemmer_wo_parser(x))
pro_descr['product_description_stemmed']=pro_descr['product_description_parsed'].map(lambda x:str_stemmer_wo_parser(x))
pro_descr['product_description_stefxmmed_woBM']=pro_descr['product_description_parsed_woBM'].map(lambda x:str_stemmer_wo_parser(x))
pro_descr['product_description_stemmed_woBrand']=pro_descr['product_description_parsed_woBrand'].map(lambda x:str_stemmer_wo_parser(x))
All_df['search_term_keys_stemmed']=All_df['search_term_keys'].map(lambda x: str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['product_title_keys_stemmed']=All_df['product_title_keys'].map(lambda x: str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['search_term_stemmed']=All_df['search_term_parsed'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['search_term_stemmed_woBM']=All_df['search_term_parsed_woBM'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['search_term_stemmed_woBrand']=All_df['search_term_parsed_woBrand'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['product_title_stemmed']=All_df['product_title_parsed'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['product_title_stemmed_woBM']=All_df['product_title_parsed_woBM'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['product_title_stemmed_woBrand']=All_df['product_title_parsed_woBrand'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['search_term_thekey_stemmed']=All_df['search_term_thekey'].map(lambda x: str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['product_title_thekey_stemmed']=All_df['product_title_thekey'].map(lambda x: str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['search_term_beforethekey_stemmed']=All_df['search_term_beforethekey'].map(lambda x: str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['product_title_beforethekey_stemmed']=All_df['product_title_beforethekey'].map(lambda x: str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['search_term_before2thekey_stemmed']=All_df['search_term_before2thekey'].map(lambda x: str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
All_df['product_title_before2thekey_stemmed']=All_df['product_title_before2thekey'].map(lambda x: str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))


In [436]:
# pro_descr = pro_descr.drop('product_description',axis=1)
All_df = pd.merge(All_df,pro_descr,how='left',on='product_uid')

In [437]:
All_df['product_description_stemmed']=All_df['product_description_parsed'].map(lambda x:str_stemmer_wo_parser(x))
All_df['product_description_stefxmmed_woBM']=All_df['product_description_parsed_woBM'].map(lambda x:str_stemmer_wo_parser(x))
All_df['product_description_stemmed_woBrand']=All_df['product_description_parsed_woBrand'].map(lambda x:str_stemmer_wo_parser(x))

# Extract features

In [443]:
for var in All_df.keys():
    All_df[var]=All_df[var].fillna("") 
    
    
def extract_after_word(s,word):
    output=""
    if word in s:
        srch= re.search(r'(?<=\b'+word+'\ )[a-zA-Z0-9\n\ \%\$\-\#\@\&\/\.\'\*\(\)\,]+',s)
        if srch!=None:
            output=srch.group(0)
    return output

In [302]:
All_df['search_term_for']=All_df['search_term_parsed'].map(lambda x: extract_after_word(x,'for'))
All_df['search_term_for_stemmed']=All_df['search_term_for'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))

All_df['search_term_with']=All_df['search_term_parsed'].map(lambda x: extract_after_word(x,'with'))
All_df['search_term_with_stemmed']=All_df['search_term_with'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))

All_df['product_title_parsed_without']=All_df['product_title_parsed'].map(lambda x: extract_after_word(x,'without'))
All_df['product_title_without_stemmed']=All_df['product_title_parsed_without'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))



In [2]:
### save the list of string variables which are not features
string_variables_list=list(All_df.keys())
print(str(len(string_variables_list) ) + " total variables...")
string_variables_list.remove('id')
string_variables_list.remove('product_uid')
string_variables_list.remove('relevance')
string_variables_list.remove('is_query_misspelled')
print("including "+ str(len(string_variables_list) ) + " text variables to drop later")

In [308]:
### STEP 1: Dummy for no attributes and empty attribute bullets
#################################################################
All_df['has_attributes_dummy']=1
All_df['has_attributes_dummy']= All_df['has_attributes_dummy'].fillna(0)
All_df['no_bullets_dummy'] = All_df['Bullet'].map(lambda x:int(len(x)==0))
All_df['is_replaced_using_google_dict']=All_df['search_term'].map(lambda x: 1 if x in google_dict.keys() else 0)

In [309]:
### STEP 2: Basic text features
#################################################################
def sentence_statistics(s):
    s= re.sub('[^a-zA-Z0-9\ \%\$\-]', '', s)
    word_list=s.split()
    meaningful_word_list=[word for word in s.split() if len(re.findall(r'\d+', word))==0 and len(wn.synsets(word))>0]
    vowels=sum([len(re.sub('[^aeiou]', '', word)) for word in word_list])
    letters = sum([len(word) for word in word_list])
    
    return len(word_list), len(meaningful_word_list), 1.0*sum([len(word) for word in word_list])/len(word_list), 1.0*vowels/letters

In [310]:
All_df['query_sentence_stats_tuple'] = All_df['search_term_parsed'].map(lambda x:  sentence_statistics(x) )
All_df['len_of_meaningful_words_in_query'] = All_df['query_sentence_stats_tuple'].map(lambda x:  x[1] )
All_df['ratio_of_meaningful_words_in_query'] = All_df['query_sentence_stats_tuple'].map(lambda x:  1.0*x[1]/x[0] )
All_df['avg_wordlength_in_query'] = All_df['query_sentence_stats_tuple'].map(lambda x:  x[2] )
All_df['ratio_vowels_in_query'] = All_df['query_sentence_stats_tuple'].map(lambda x:  x[3] )
All_df=All_df.drop(['query_sentence_stats_tuple'],axis=1)

In [447]:
All_df['initial_len_of_query'] = All_df['search_term_stemmed'].map(lambda x:len(x.split()))#.astype(np.int64)
All_df['len_of_query_woBM'] = All_df['search_term_stemmed_woBM'].map(lambda x:len(x.split()))#.astype(np.int64)

All_df['len_of_query_for'] = All_df['search_term_for_stemmed'].map(lambda x:len(words_wo_digits(x, minLength=1).split()))#.astype(np.int64)
All_df['len_of_query_with'] = All_df['search_term_with_stemmed'].map(lambda x:len(words_wo_digits(x, minLength=1).split()))#.astype(np.int64)
All_df['len_of_prtitle_without'] = All_df['product_title_without_stemmed'].map(lambda x:len(words_wo_digits(x, minLength=1).split()))#.astype(np.int64)

All_df['len_of_query_string_only_woBM'] = All_df['search_term_stemmed_woBM'].map(lambda x:len(words_wo_digits(x, minLength=1).split()))#.astype(np.int64)

All_df['len_of_query_w_dash_woBM'] = All_df['search_term_stemmed_woBM'].map(lambda x:len(words_w_dash(x).split()))#.astype(np.int64)
All_df['len_of_title_w_dash_woBM'] = All_df['product_title_stemmed_woBM'].map(lambda x:len(words_w_dash(x).split()))#.astype(np.int64)

All_df['len_of_product_title_woBM'] = All_df['product_title_stemmed_woBM'].map(lambda x:len(x.split()))#.astype(np.int64)
################# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ###############
All_df['len_of_product_description_woBM'] = All_df['product_description_stefxmmed_woBM'].map(lambda x:len(x.split())).astype(np.int64)
#############################################################################
All_df['len_of_attribute_bullets_woBM'] = All_df['attribute_bullets_stemmed_woBM'].map(lambda x:len(x.split()))#.astype(np.int64)

In [324]:
All_df['len_of_brands_in_query'] = All_df['brands_in_search_term'].map(lambda x:len(x.split())).astype(np.int64)
All_df['size_of_brands_in_query'] = All_df['brands_in_search_term'].map(lambda x:len(x.split(";"))).astype(np.int64)
All_df['len_of_materials_in_query'] = All_df['materials_in_search_term'].map(lambda x:len(x.split())).astype(np.int64)
All_df['size_of_materials_in_query'] = All_df['materials_in_search_term'].map(lambda x:len(x.split(";"))).astype(np.int64)

In [325]:
All_df['len_of_brands_in_product_title'] = All_df['brands_in_product_title'].map(lambda x:len(x.split())).astype(np.int64)
All_df['size_of_brands_in_product_title'] = All_df['brands_in_product_title'].map(lambda x:len(x.split(";"))).astype(np.int64)
All_df['len_of_materials_in_product_title'] = All_df['materials_in_product_title'].map(lambda x:len(x.split())).astype(np.int64)
All_df['size_of_materials_in_product_title'] = All_df['materials_in_product_title'].map(lambda x:len(x.split(";"))).astype(np.int64)

In [448]:
All_df['len_of_brands_in_product_description'] = All_df['brands_in_product_description'].map(lambda x:len(x.split())).astype(np.int64)
All_df['size_of_brands_in_product_description'] = All_df['brands_in_product_description'].map(lambda x:len(x.split(";"))).astype(np.int64)
All_df['len_of_materials_in_product_description'] = All_df['materials_in_product_description'].map(lambda x:len(x.split())).astype(np.int64)
All_df['size_of_materials_in_product_description'] = All_df['materials_in_product_description'].map(lambda x:len(x.split(";"))).astype(np.int64)

In [328]:
All_df['len_of_brands_in_attribute_bullets'] = All_df['brands_in_attribute_bullets'].map(lambda x:len(x.split())).astype(np.int64)
All_df['size_of_brands_in_attribute_bullets'] = All_df['brands_in_attribute_bullets'].map(lambda x:len(x.split(";"))).astype(np.int64)
All_df['len_of_materials_in_attribute_bullets'] = All_df['materials_in_attribute_bullets'].map(lambda x:len(x.split())).astype(np.int64)
All_df['size_of_materials_in_attribute_bullets'] = All_df['materials_in_attribute_bullets'].map(lambda x:len(x.split(";"))).astype(np.int64)


In [449]:
All_df['len_of_query']=All_df['len_of_query_woBM']+All_df['size_of_brands_in_query']+All_df['size_of_materials_in_query']
All_df['len_of_product_title']=All_df['len_of_product_title_woBM']+All_df['size_of_brands_in_product_title']
All_df['len_of_product_description']=All_df['len_of_product_title_woBM']+All_df['size_of_brands_in_product_description']+All_df['size_of_materials_in_product_description']

In [331]:
All_df['len_of_attribute_bullets']=All_df['len_of_attribute_bullets_woBM']+All_df['size_of_brands_in_attribute_bullets']+All_df['size_of_materials_in_attribute_bullets']

All_df['len_of_query_keys'] = All_df['search_term_keys_stemmed'].map(lambda x:len(x.split()))#.astype(np.int64)
All_df['len_of_product_title_keys'] = All_df['product_title_keys_stemmed'].map(lambda x:len(x.split()))#.astype(np.int64)
All_df['len_of_query_thekey'] = All_df['search_term_thekey'].map(lambda x:len(x.split()))#.astype(np.int64)
All_df['len_of_product_title_thekey'] = All_df['product_title_thekey'].map(lambda x:len(x.split()))#.astype(np.int64)
All_df['len_of_query_beforethekey'] = All_df['search_term_beforethekey'].map(lambda x:len(x.split()))#.astype(np.int64)
All_df['len_of_product_title_beforethekey'] = All_df['product_title_beforethekey'].map(lambda x:len(x.split()))#.astype(np.int64)
All_df['len_of_query_before2thekey'] = All_df['search_term_before2thekey'].map(lambda x:len(x.split()))#.astype(np.int64)
All_df['len_of_product_title_before2thekey'] = All_df['product_title_before2thekey'].map(lambda x:len(x.split()))#.astype(np.int64)


In [332]:
def find_ratio(a,b):
    if b==0:
        return 0
    else:
        return min(1,1.0*a/b)


def perc_digits_in_str(s):
    if len(s.split())==0:
        output=0
    else:
        output =1.0 -1.0*len(words_wo_digits(s, minLength=1).split()) / len(s.split())
    return output

In [412]:
All_df['attribute_bullets_parsed'] = All_df['attribute_bullets_parsed'].fillna('')

In [450]:
for column_name in ['search_term', 'product_title', 'product_description','attribute_bullets']:
    All_df['ratio_of_nn_important_in_'+column_name]= All_df.apply(lambda x: \
        find_ratio(len(nn_important_words(x['search_term_tokens']).split()), len(x[column_name+'_parsed'])) ,axis=1)
    All_df['ratio_of_nn_unimportant_in_'+column_name]= All_df.apply(lambda x: \
        find_ratio(len(nn_unimportant_words(x['search_term_tokens']).split()), len(x[column_name+'_parsed'])) ,axis=1)
    All_df['ratio_of_vb_in_'+column_name]= All_df.apply(lambda x: \
        find_ratio(len(vb_words(x['search_term_tokens']).split()), len(x[column_name+'_parsed'])) ,axis=1)
    All_df['ratio_of_vbg_in_'+column_name]= All_df.apply(lambda x: \
        find_ratio(len(vbg_words(x['search_term_tokens']).split()), len(x[column_name+'_parsed'])) ,axis=1)
    All_df['ratio_of_jj_rb_in_'+column_name]= All_df.apply(lambda x: \
        find_ratio(len(nn_unimportant_words(x['search_term_tokens']).split()), len(x[column_name+'_parsed'])) ,axis=1)
    
        
print('len_of_something time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

All_df['perc_digits_in_query'] = All_df['search_term_stemmed'].map(lambda x: perc_digits_in_str(x)  )
All_df['perc_digits_in_title'] = All_df['product_title_stemmed'].map(lambda x: perc_digits_in_str(x)  )
All_df['perc_digits_in_description'] = All_df['product_description_stemmed'].map(lambda x: perc_digits_in_str(x) )
All_df['perc_digits_in_bullets'] = All_df['attribute_bullets_stemmed'].map(lambda x: perc_digits_in_str(x) )

print('perc_digits time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

len_of_something time: 91.7 minutes

perc_digits time: 1.3 minutes



In [451]:
def query_brand_material_in_attribute(str_query_brands,str_attribute_brands):
    list_query_brands=list(set(str_query_brands.split(";")))
    list_attribute_brands=list(set(str_attribute_brands.split(";")))
    while '' in list_query_brands:
        list_query_brands.remove('')
    while '' in list_attribute_brands:
        list_attribute_brands.remove('')
    
    str_attribute_brands=" ".join(str_attribute_brands.split(";"))    
    full_match=0
    partial_match=0
    assumed_match=0
    no_match=0
    num_of_query_brands=len(list_query_brands)
    num_of_attribute_brands=len(list_attribute_brands)
    if num_of_query_brands>0:
        for brand in list_query_brands:
            if brand in list_attribute_brands:
                full_match+=1
            elif ' '+brand+' ' in ' '+str_attribute_brands+' ':
                partial_match+=1
            elif (' '+brand.split()[0] in ' '+str_attribute_brands and brand.split()[0][0] not in "0123456789") or \
            (len(brand.split())>1 and (' '+brand.split()[0]+' '+brand.split()[1]) in ' '+str_attribute_brands):
                assumed_match+=1
            else:
                no_match+=1
                
    convoluted_output=0 # no brand in query
    if num_of_query_brands>0:
        if num_of_attribute_brands==0:
            convoluted_output = -1 # no brand in text, but there is brand in query
        elif no_match==0:
            if assumed_match==0:
                convoluted_output=3 # all brands fully matched
            else:
                convoluted_output=2 # all brands matched at least partially
        else:
            if full_match+ partial_match+ assumed_match>0:
                convoluted_output = 1 # one brand matched but the other is not
            else:
                convoluted_output= -2  #brand mismatched
    return full_match, partial_match, assumed_match, no_match, convoluted_output

In [452]:
### brand 
All_df['brands_all']=All_df['brands_in_search_term']+"\t"+All_df['brands_in_product_title']+"\t"+All_df['brands_in_product_description']+"\t"\
+All_df['brands_in_attribute_bullets']+"\t"+All_df['brand_parsed']

In [453]:
All_df['query_brand_in_brand_tuple']=All_df['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[4]))
All_df['query_brand_in_brand_fullmatch']=All_df['query_brand_in_brand_tuple'].map(lambda x: x[0])
All_df['query_brand_in_brand_partialmatch']=All_df['query_brand_in_brand_tuple'].map(lambda x: x[1])
All_df['query_brand_in_brand_assumedmatch']=All_df['query_brand_in_brand_tuple'].map(lambda x: x[2])
All_df['query_brand_in_brand_nomatch']=All_df['query_brand_in_brand_tuple'].map(lambda x: x[3])
All_df['query_brand_in_brand_convoluted']=All_df['query_brand_in_brand_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_brand_in_brand_tuple'],axis=1)

In [454]:
All_df['query_brand_in_all_tuple']=All_df['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],";".join(x.split("\t")[1:])))
All_df['query_brand_in_all_fullmatch']=All_df['query_brand_in_all_tuple'].map(lambda x: x[0])
All_df['query_brand_in_all_partialmatch']=All_df['query_brand_in_all_tuple'].map(lambda x: x[1])
All_df['query_brand_in_all_assumedmatch']=All_df['query_brand_in_all_tuple'].map(lambda x: x[2])
All_df['query_brand_in_all_nomatch']=All_df['query_brand_in_all_tuple'].map(lambda x: x[3])
All_df['query_brand_in_all_convoluted']=All_df['query_brand_in_all_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_brand_in_all_tuple'],axis=1)

In [455]:
All_df['query_brand_in_title_tuple']=All_df['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[1]))
All_df['query_brand_in_title_fullmatch']=All_df['query_brand_in_title_tuple'].map(lambda x: x[0])
All_df['query_brand_in_title_partialmatch']=All_df['query_brand_in_title_tuple'].map(lambda x: x[1])
All_df['query_brand_in_title_assumedmatch']=All_df['query_brand_in_title_tuple'].map(lambda x: x[2])
All_df['query_brand_in_title_nomatch']=All_df['query_brand_in_title_tuple'].map(lambda x: x[3])
All_df['query_brand_in_title_convoluted']=All_df['query_brand_in_title_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_brand_in_title_tuple'],axis=1)

In [456]:
All_df['query_brand_in_description_tuple']=All_df['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[2]))
All_df['query_brand_in_description_convoluted']=All_df['query_brand_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_brand_in_description_tuple'],axis=1)

All_df['query_brand_in_bullets_tuple']=All_df['brands_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[3]))
All_df['query_brand_in_bullets_convoluted']=All_df['query_brand_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_brand_in_bullets_tuple'],axis=1)

All_df=All_df.drop(['brands_all'],axis=1)

print('create brand match variables time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

create brand match variables time: 4.0 minutes



In [457]:
### material
All_df['materials_all']=All_df['materials_in_search_term']+"\t"+All_df['materials_in_product_title']+"\t"+\
All_df['materials_in_product_description']+"\t"+All_df['materials_in_attribute_bullets']+"\t"+All_df['material_parsed']

In [458]:
All_df['query_material_in_material_tuple']=All_df['materials_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[4]))
All_df['query_material_in_material_fullmatch']=All_df['query_material_in_material_tuple'].map(lambda x: x[0])
All_df['query_material_in_material_partialmatch']=All_df['query_material_in_material_tuple'].map(lambda x: x[1])
All_df['query_material_in_material_assumedmatch']=All_df['query_material_in_material_tuple'].map(lambda x: x[2])
All_df['query_material_in_material_nomatch']=All_df['query_material_in_material_tuple'].map(lambda x: x[3])
All_df['query_material_in_material_convoluted']=All_df['query_material_in_material_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_material_in_material_tuple'],axis=1)

In [459]:
All_df['query_material_in_all_tuple']=All_df['materials_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],";".join(x.split("\t")[1:])))
All_df['query_material_in_all_fullmatch']=All_df['query_material_in_all_tuple'].map(lambda x: x[0])
All_df['query_material_in_all_partialmatch']=All_df['query_material_in_all_tuple'].map(lambda x: x[1])
All_df['query_material_in_all_assumedmatch']=All_df['query_material_in_all_tuple'].map(lambda x: x[2])
All_df['query_material_in_all_nomatch']=All_df['query_material_in_all_tuple'].map(lambda x: x[3])
All_df['query_material_in_all_convoluted']=All_df['query_material_in_all_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_material_in_all_tuple'],axis=1)

In [460]:
All_df['query_material_in_title_tuple']=All_df['materials_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[1]))
All_df['query_material_in_title_convoluted']=All_df['query_material_in_title_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_material_in_title_tuple'],axis=1)

All_df['query_material_in_description_tuple']=All_df['materials_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[2]))
All_df['query_material_in_description_convoluted']=All_df['query_material_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_material_in_description_tuple'],axis=1)

All_df['query_material_in_bullets_tuple']=All_df['materials_all'].map(lambda x: query_brand_material_in_attribute(x.split("\t")[0],x.split("\t")[3]))
All_df['query_material_in_bullets_convoluted']=All_df['query_material_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['query_material_in_bullets_tuple'],axis=1)

All_df=All_df.drop(['materials_all'],axis=1)

print('create material match variables time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

create material match variables time: 0.2 minutes



In [461]:
###Count number of similar words/letters/bigrams etc
### query vs product title
All_df['wordFor_in_title_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_for_stemmed'],x['product_title_stemmed'],string_only=True),axis=1)
All_df['wordFor_in_title_string_only_num'] = All_df['wordFor_in_title_string_only_tuple'].map(lambda x: x[0])
All_df['wordFor_in_title_string_only_let'] = All_df['wordFor_in_title_string_only_tuple'].map(lambda x: x[2])
All_df['wordFor_in_title_string_only_letratio'] = All_df['wordFor_in_title_string_only_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['wordFor_in_title_string_only_tuple'],axis=1)

All_df['wordWith_in_title_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_with_stemmed'],x['product_title_stemmed'],string_only=True),axis=1)
All_df['wordWith_in_title_string_only_num'] = All_df['wordWith_in_title_string_only_tuple'].map(lambda x: x[0])
All_df['wordWith_in_title_string_only_let'] = All_df['wordWith_in_title_string_only_tuple'].map(lambda x: x[2])
All_df['wordWith_in_title_string_only_letratio'] = All_df['wordWith_in_title_string_only_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['wordWith_in_title_string_only_tuple'],axis=1)

In [462]:
All_df['prtitleWithout_in_query_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['product_title_without_stemmed'],x['search_term_stemmed'],string_only=True),axis=1)
All_df['prtitleWithout_in_query_string_only_num'] = All_df['prtitleWithout_in_query_string_only_tuple'].map(lambda x: x[0])
All_df['prtitleWithout_in_query_string_only_let'] = All_df['prtitleWithout_in_query_string_only_tuple'].map(lambda x: x[2])
All_df['prtitleWithout_in_query_string_only_letratio'] = All_df['prtitleWithout_in_query_string_only_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['prtitleWithout_in_query_string_only_tuple'],axis=1)

All_df['query_in_title']=All_df.apply(lambda x: \
            query_in_text(x['search_term_stemmed'],x['product_title_stemmed']),axis=1)

All_df['word_in_title_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_stemmed'],x['product_title_stemmed']),axis=1)
All_df['word_in_title_num'] = All_df['word_in_title_tuple'].map(lambda x: x[0])
All_df['word_in_title_sum'] = All_df['word_in_title_tuple'].map(lambda x: x[1])
All_df['word_in_title_let'] = All_df['word_in_title_tuple'].map(lambda x: x[2])
All_df['word_in_title_numratio'] = All_df['word_in_title_tuple'].map(lambda x: x[3])
All_df['word_in_title_letratio'] = All_df['word_in_title_tuple'].map(lambda x: x[4])
All_df['word_in_title_string'] = All_df['word_in_title_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['word_in_title_tuple'],axis=1)

In [463]:
All_df['word_in_title_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_stemmed'],x['product_title_stemmed'],string_only=True),axis=1)
All_df['word_in_title_string_only_num'] = All_df['word_in_title_string_only_tuple'].map(lambda x: x[0])
All_df['word_in_title_string_only_sum'] = All_df['word_in_title_string_only_tuple'].map(lambda x: x[1])
All_df['word_in_title_string_only_let'] = All_df['word_in_title_string_only_tuple'].map(lambda x: x[2])
All_df['word_in_title_string_only_numratio'] = All_df['word_in_title_string_only_tuple'].map(lambda x: x[3])
All_df['word_in_title_string_only_letratio'] = All_df['word_in_title_string_only_tuple'].map(lambda x: x[4])
All_df['word_in_title_string_only_string'] = All_df['word_in_title_string_only_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['word_in_title_string_only_tuple'],axis=1)

In [464]:
All_df['word_in_title_w_dash']=All_df.apply(lambda x: \
            str_common_word(words_w_dash(x['search_term_stemmed']),words_w_dash(x['product_title_stemmed']))[0],axis=1)

In [516]:
def str_2common_words_111(str1, str2, string_only=False):
    num=0
    total_entries=0
    cnt_letters=0
    cnt_unique_letters=0
    words_in_query=str1.split()
    for cnt in range(0,len(words_in_query)-1):
        two_words=words_in_query[cnt]+' '+words_in_query[cnt+1]
        if string_only==False or len(re.findall(r'\d+', two_words))==0:
            if (' '+two_words+' ') in (' '+str2+' '):
                num+=1
                total_entries+=(' '+str2+' ').count(' '+two_words+' ')
                cnt_letters+=(' '+str2+' ').count(' '+two_words+' ') * (len(two_words)-1)
                cnt_unique_letters+=(len(two_words)-1)
    return num, total_entries, cnt_unique_letters

In [519]:
All_df['two_words_in_title_tuple']=All_df.apply(lambda x: \
            str_2common_words_111(x['search_term_stemmed'],x['product_title_stemmed']),axis=1)
All_df['two_words_in_title_num'] = All_df['two_words_in_title_tuple'].map(lambda x: x[0])
All_df['two_words_in_title_sum'] = All_df['two_words_in_title_tuple'].map(lambda x: x[1])
All_df['two_words_in_title_let'] = All_df['two_words_in_title_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['two_words_in_title_tuple'],axis=1)

In [520]:
All_df['two_words_in_title_string_only_tuple']=All_df.apply(lambda x: \
            str_2common_words_111(x['search_term_stemmed'],x['product_title_stemmed'],string_only=True),axis=1)
All_df['two_words_in_title_string_only_num'] = All_df['two_words_in_title_string_only_tuple'].map(lambda x: x[0])
All_df['two_words_in_title_string_only_sum'] = All_df['two_words_in_title_string_only_tuple'].map(lambda x: x[1])
All_df['two_words_in_title_string_only_let'] = All_df['two_words_in_title_string_only_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['two_words_in_title_string_only_tuple'],axis=1)

In [521]:
All_df['common_digits_in_title_tuple']=All_df.apply(lambda x: \
            str_common_digits(x['search_term_stemmed'],x['product_title_stemmed']),axis=1)
All_df['len_of_digits_in_query'] = All_df['common_digits_in_title_tuple'].map(lambda x: x[0])
All_df['len_of_digits_in_title'] = All_df['common_digits_in_title_tuple'].map(lambda x: x[1])
All_df['common_digits_in_title_num'] = All_df['common_digits_in_title_tuple'].map(lambda x: x[2])
All_df['common_digits_in_title_ratio'] = All_df['common_digits_in_title_tuple'].map(lambda x: x[3])
All_df['common_digits_in_title_jaccard'] = All_df['common_digits_in_title_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['common_digits_in_title_tuple'],axis=1)

In [523]:
All_df['nn_important_in_title_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            x['product_title_stemmed']),axis=1)
All_df['nn_important_in_title_num'] = All_df['nn_important_in_title_tuple'].map(lambda x: x[0])
All_df['nn_important_in_title_sum'] = All_df['nn_important_in_title_tuple'].map(lambda x: x[1])
All_df['nn_important_in_title_let'] = All_df['nn_important_in_title_tuple'].map(lambda x: x[2])
All_df['nn_important_in_title_numratio'] = All_df['nn_important_in_title_tuple'].map(lambda x: x[3])
All_df['nn_important_in_title_letratio'] = All_df['nn_important_in_title_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_title_tuple'],axis=1)

In [524]:
All_df['nn_unimportant_in_title_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_unimportant_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            x['product_title_stemmed']),axis=1)
All_df['nn_unimportant_in_title_num'] = All_df['nn_unimportant_in_title_tuple'].map(lambda x: x[0])
All_df['nn_unimportant_in_title_let'] = All_df['nn_unimportant_in_title_tuple'].map(lambda x: x[2])
All_df['nn_unimportant_in_title_letratio'] = All_df['nn_unimportant_in_title_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_unimportant_in_title_tuple'],axis=1)

In [525]:
All_df['nn_important_in_nn_important_in_title_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_important_words(x['product_title_tokens']),stoplist=stoplist_wo_can)),axis=1)
All_df['nn_important_in_nn_important_in_title_num'] = All_df['nn_important_in_nn_important_in_title_tuple'].map(lambda x: x[0])
All_df['nn_important_in_nn_important_in_title_sum'] = All_df['nn_important_in_nn_important_in_title_tuple'].map(lambda x: x[1])
All_df['nn_important_in_nn_important_in_title_let'] = All_df['nn_important_in_nn_important_in_title_tuple'].map(lambda x: x[2])
All_df['nn_important_in_nn_important_in_title_letratio'] = All_df['nn_important_in_nn_important_in_title_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_nn_important_in_title_tuple'],axis=1)

In [526]:
All_df['nn_important_in_nn_unimportant_in_title_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_unimportant_words(x['product_title_tokens']),stoplist=stoplist_wo_can)),axis=1)
All_df['nn_important_in_nn_unimportant_in_title_num'] = All_df['nn_important_in_nn_unimportant_in_title_tuple'].map(lambda x: x[0])
All_df['nn_important_in_nn_unimportant_in_title_sum'] = All_df['nn_important_in_nn_unimportant_in_title_tuple'].map(lambda x: x[1])
All_df['nn_important_in_nn_unimportant_in_title_let'] = All_df['nn_important_in_nn_unimportant_in_title_tuple'].map(lambda x: x[2])
All_df['nn_important_in_nn_unimportant_in_title_letratio'] = All_df['nn_important_in_nn_unimportant_in_title_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_nn_unimportant_in_title_tuple'],axis=1)

In [527]:
All_df['nn_unimportant_in_nn_important_in_title_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_unimportant_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_important_words(x['product_title_tokens']),stoplist=stoplist_wo_can)),axis=1)
All_df['nn_unimportant_in_nn_important_in_title_num'] = All_df['nn_unimportant_in_nn_important_in_title_tuple'].map(lambda x: x[0])
All_df['nn_unimportant_in_nn_important_in_title_sum'] = All_df['nn_unimportant_in_nn_important_in_title_tuple'].map(lambda x: x[1])
All_df['nn_unimportant_in_nn_important_in_title_let'] = All_df['nn_unimportant_in_nn_important_in_title_tuple'].map(lambda x: x[2])
All_df['nn_unimportant_in_nn_important_in_title_letratio'] = All_df['nn_unimportant_in_nn_important_in_title_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_unimportant_in_nn_important_in_title_tuple'],axis=1)

In [528]:
All_df['jj_rb_in_jj_rb_in_title_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(jj_rb_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(jj_rb_words(x['product_title_tokens']),stoplist=stoplist_wo_can)),axis=1)
All_df['jj_rb_in_jj_rb_in_title_num'] = All_df['jj_rb_in_jj_rb_in_title_tuple'].map(lambda x: x[0])
All_df['jj_rb_in_jj_rb_in_title_sum'] = All_df['jj_rb_in_jj_rb_in_title_tuple'].map(lambda x: x[1])
All_df['jj_rb_in_jj_rb_in_title_let'] = All_df['jj_rb_in_jj_rb_in_title_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['jj_rb_in_jj_rb_in_title_tuple'],axis=1)

In [529]:
All_df['vbg_in_vbg_in_title_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(vbg_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(vbg_words(x['product_title_tokens']),stoplist=stoplist_wo_can)),axis=1)
All_df['vbg_in_vbg_in_title_num'] = All_df['vbg_in_vbg_in_title_tuple'].map(lambda x: x[0])
All_df['vbg_in_vbg_in_title_sum'] = All_df['vbg_in_vbg_in_title_tuple'].map(lambda x: x[1])
All_df['vbg_in_vbg_in_title_let'] = All_df['vbg_in_vbg_in_title_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['vbg_in_vbg_in_title_tuple'],axis=1)
print('words_in_title time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

words_in_title time: 98.9 minutes



In [530]:
### query vs product description
All_df['wordFor_in_description_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_for_stemmed'],x['product_description_stemmed'],string_only=True),axis=1)
All_df['wordFor_in_description_string_only_num'] = All_df['wordFor_in_description_string_only_tuple'].map(lambda x: x[0])
All_df['wordFor_in_description_string_only_let'] = All_df['wordFor_in_description_string_only_tuple'].map(lambda x: x[2])
All_df['wordFor_in_description_string_only_letratio'] = All_df['wordFor_in_description_string_only_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['wordFor_in_description_string_only_tuple'],axis=1)


In [531]:
All_df['wordWith_in_description_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_with_stemmed'],x['product_description_stemmed'],string_only=True),axis=1)
All_df['wordWith_in_description_string_only_num'] = All_df['wordWith_in_description_string_only_tuple'].map(lambda x: x[0])
All_df['wordWith_in_description_string_only_let'] = All_df['wordWith_in_description_string_only_tuple'].map(lambda x: x[2])
All_df['wordWith_in_description_string_only_letratio'] = All_df['wordWith_in_description_string_only_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['wordWith_in_description_string_only_tuple'],axis=1)

In [532]:
All_df['query_in_description']=All_df.apply(lambda x: \
            query_in_text(x['search_term_stemmed'],x['product_description_stemmed']),axis=1)

All_df['word_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_stemmed'],x['product_description_stemmed']),axis=1)
All_df['word_in_description_num'] = All_df['word_in_description_tuple'].map(lambda x: x[0])
All_df['word_in_description_sum'] = All_df['word_in_description_tuple'].map(lambda x: x[1])
All_df['word_in_description_let'] = All_df['word_in_description_tuple'].map(lambda x: x[2])
All_df['word_in_description_numratio'] = All_df['word_in_description_tuple'].map(lambda x: x[3])
All_df['word_in_description_letratio'] = All_df['word_in_description_tuple'].map(lambda x: x[4])
All_df['word_in_description_string'] = All_df['word_in_description_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['word_in_description_tuple'],axis=1)

In [533]:
All_df['word_in_description_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_stemmed'],x['product_description_stemmed'],string_only=True),axis=1)
All_df['word_in_description_string_only_num'] = All_df['word_in_description_string_only_tuple'].map(lambda x: x[0])
All_df['word_in_description_string_only_sum'] = All_df['word_in_description_string_only_tuple'].map(lambda x: x[1])
All_df['word_in_description_string_only_let'] = All_df['word_in_description_string_only_tuple'].map(lambda x: x[2])
All_df['word_in_description_string_only_numratio'] = All_df['word_in_description_string_only_tuple'].map(lambda x: x[3])
All_df['word_in_description_string_only_letratio'] = All_df['word_in_description_string_only_tuple'].map(lambda x: x[4])
All_df['word_in_description_string_only_string'] = All_df['word_in_description_string_only_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['word_in_description_string_only_tuple'],axis=1)


In [535]:
All_df['word_in_description_w_dash']=All_df.apply(lambda x: \
            str_common_word(words_w_dash(x['search_term_stemmed']),words_w_dash(x['product_description_stemmed']))[0],axis=1)

All_df['two_words_in_description_tuple']=All_df.apply(lambda x: \
            str_2common_words_111(x['search_term_stemmed'],x['product_description_stemmed']),axis=1)
All_df['two_words_in_description_num'] = All_df['two_words_in_description_tuple'].map(lambda x: x[0])
All_df['two_words_in_description_sum'] = All_df['two_words_in_description_tuple'].map(lambda x: x[1])
All_df['two_words_in_description_let'] = All_df['two_words_in_description_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['two_words_in_description_tuple'],axis=1)

In [536]:
All_df['two_words_in_description_string_only_tuple']=All_df.apply(lambda x: \
            str_2common_words_111(x['search_term_stemmed'],x['product_description_stemmed'],string_only=True),axis=1)
All_df['two_words_in_description_string_only_num'] = All_df['two_words_in_description_string_only_tuple'].map(lambda x: x[0])
All_df['two_words_in_description_string_only_sum'] = All_df['two_words_in_description_string_only_tuple'].map(lambda x: x[1])
All_df['two_words_in_description_string_only_let'] = All_df['two_words_in_description_string_only_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['two_words_in_description_string_only_tuple'],axis=1)

In [537]:
All_df['common_digits_in_description_tuple']=All_df.apply(lambda x: \
            str_common_digits(x['search_term_stemmed'],x['product_description_stemmed']),axis=1)
All_df['len_of_digits_in_query'] = All_df['common_digits_in_description_tuple'].map(lambda x: x[0])
All_df['len_of_digits_in_description'] = All_df['common_digits_in_description_tuple'].map(lambda x: x[1])
All_df['common_digits_in_description_num'] = All_df['common_digits_in_description_tuple'].map(lambda x: x[2])
All_df['common_digits_in_description_ratio'] = All_df['common_digits_in_description_tuple'].map(lambda x: x[3])
All_df['common_digits_in_description_jaccard'] = All_df['common_digits_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['common_digits_in_description_tuple'],axis=1)


In [538]:
All_df['nn_important_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            x['product_description_stemmed']),axis=1)
All_df['nn_important_in_description_num'] = All_df['nn_important_in_description_tuple'].map(lambda x: x[0])
All_df['nn_important_in_description_sum'] = All_df['nn_important_in_description_tuple'].map(lambda x: x[1])
All_df['nn_important_in_description_let'] = All_df['nn_important_in_description_tuple'].map(lambda x: x[2])
All_df['nn_important_in_description_numratio'] = All_df['nn_important_in_description_tuple'].map(lambda x: x[3])
All_df['nn_important_in_description_letratio'] = All_df['nn_important_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_description_tuple'],axis=1)

In [539]:
All_df['nn_unimportant_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_unimportant_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            x['product_description_stemmed']),axis=1)
All_df['nn_unimportant_in_description_num'] = All_df['nn_unimportant_in_description_tuple'].map(lambda x: x[0])
All_df['nn_unimportant_in_description_let'] = All_df['nn_unimportant_in_description_tuple'].map(lambda x: x[2])
All_df['nn_unimportant_in_description_letratio'] = All_df['nn_unimportant_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_unimportant_in_description_tuple'],axis=1)

In [566]:
All_df['nn_important_in_nn_important_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_important_words(str(x['product_description_tokens'])))),axis=1)
All_df['nn_important_in_nn_important_in_description_num'] = All_df['nn_important_in_nn_important_in_description_tuple'].map(lambda x: x[0])
All_df['nn_important_in_nn_important_in_description_sum'] = All_df['nn_important_in_nn_important_in_description_tuple'].map(lambda x: x[1])
All_df['nn_important_in_nn_important_in_description_let'] = All_df['nn_important_in_nn_important_in_description_tuple'].map(lambda x: x[2])
All_df['nn_important_in_nn_important_in_description_letratio'] = All_df['nn_important_in_nn_important_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_nn_important_in_description_tuple'],axis=1)

In [567]:
All_df['nn_important_in_nn_unimportant_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_unimportant_words(str(x['product_description_tokens'])))),axis=1)
All_df['nn_important_in_nn_unimportant_in_description_num'] = All_df['nn_important_in_nn_unimportant_in_description_tuple'].map(lambda x: x[0])
All_df['nn_important_in_nn_unimportant_in_description_sum'] = All_df['nn_important_in_nn_unimportant_in_description_tuple'].map(lambda x: x[1])
All_df['nn_important_in_nn_unimportant_in_description_let'] = All_df['nn_important_in_nn_unimportant_in_description_tuple'].map(lambda x: x[2])
All_df['nn_important_in_nn_unimportant_in_description_letratio'] = All_df['nn_important_in_nn_unimportant_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_nn_unimportant_in_description_tuple'],axis=1)

In [570]:
All_df['nn_unimportant_in_nn_important_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_unimportant_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_important_words(str(x['product_description_tokens'])))),axis=1)
All_df['nn_unimportant_in_nn_important_in_description_num'] = All_df['nn_unimportant_in_nn_important_in_description_tuple'].map(lambda x: x[0])
All_df['nn_unimportant_in_nn_important_in_description_sum'] = All_df['nn_unimportant_in_nn_important_in_description_tuple'].map(lambda x: x[1])
All_df['nn_unimportant_in_nn_important_in_description_let'] = All_df['nn_unimportant_in_nn_important_in_description_tuple'].map(lambda x: x[2])
All_df['nn_unimportant_in_nn_important_in_description_letratio'] = All_df['nn_unimportant_in_nn_important_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_unimportant_in_nn_important_in_description_tuple'],axis=1)

In [573]:
All_df['jj_rb_in_jj_rb_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(jj_rb_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(jj_rb_words(str(x['product_description_tokens'])))),axis=1)
All_df['jj_rb_in_jj_rb_in_description_num'] = All_df['jj_rb_in_jj_rb_in_description_tuple'].map(lambda x: x[0])
All_df['jj_rb_in_jj_rb_in_description_sum'] = All_df['jj_rb_in_jj_rb_in_description_tuple'].map(lambda x: x[1])
All_df['jj_rb_in_jj_rb_in_description_let'] = All_df['jj_rb_in_jj_rb_in_description_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['jj_rb_in_jj_rb_in_description_tuple'],axis=1)

In [574]:
All_df['vbg_in_vbg_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(vbg_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(vbg_words(str(x['product_description_tokens'])))),axis=1)
All_df['vbg_in_vbg_in_description_num'] = All_df['vbg_in_vbg_in_description_tuple'].map(lambda x: x[0])
All_df['vbg_in_vbg_in_description_sum'] = All_df['vbg_in_vbg_in_description_tuple'].map(lambda x: x[1])
All_df['vbg_in_vbg_in_description_let'] = All_df['vbg_in_vbg_in_description_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['vbg_in_vbg_in_description_tuple'],axis=1)
print('words_in_description time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()


words_in_description time: 43.9 minutes



In [575]:
### query vs attribute bullets
All_df['wordFor_in_bullets_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_for_stemmed'],x['attribute_bullets_stemmed'],string_only=True),axis=1)
All_df['wordFor_in_bullets_string_only_num'] = All_df['wordFor_in_bullets_string_only_tuple'].map(lambda x: x[0])
All_df['wordFor_in_bullets_string_only_let'] = All_df['wordFor_in_bullets_string_only_tuple'].map(lambda x: x[2])
All_df['wordFor_in_bullets_string_only_letratio'] = All_df['wordFor_in_bullets_string_only_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['wordFor_in_bullets_string_only_tuple'],axis=1)

In [576]:
All_df['wordWith_in_bullets_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_with_stemmed'],x['attribute_bullets_stemmed'],string_only=True),axis=1)
All_df['wordWith_in_bullets_string_only_num'] = All_df['wordWith_in_bullets_string_only_tuple'].map(lambda x: x[0])
All_df['wordWith_in_bullets_string_only_let'] = All_df['wordWith_in_bullets_string_only_tuple'].map(lambda x: x[2])
All_df['wordWith_in_bullets_string_only_letratio'] = All_df['wordWith_in_bullets_string_only_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['wordWith_in_bullets_string_only_tuple'],axis=1)

In [577]:
All_df['query_in_bullets']=All_df.apply(lambda x: \
            query_in_text(x['search_term_stemmed'],x['attribute_bullets_stemmed']),axis=1)

All_df['word_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_stemmed'],x['attribute_bullets_stemmed']),axis=1)
All_df['word_in_bullets_num'] = All_df['word_in_bullets_tuple'].map(lambda x: x[0])
All_df['word_in_bullets_sum'] = All_df['word_in_bullets_tuple'].map(lambda x: x[1])
All_df['word_in_bullets_let'] = All_df['word_in_bullets_tuple'].map(lambda x: x[2])
All_df['word_in_bullets_numratio'] = All_df['word_in_bullets_tuple'].map(lambda x: x[3])
All_df['word_in_bullets_letratio'] = All_df['word_in_bullets_tuple'].map(lambda x: x[4])
All_df['word_in_bullets_string'] = All_df['word_in_bullets_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['word_in_bullets_tuple'],axis=1)

In [578]:
All_df['word_in_bullets_string_only_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_stemmed'],x['attribute_bullets_stemmed'],string_only=True),axis=1)
All_df['word_in_bullets_string_only_num'] = All_df['word_in_bullets_string_only_tuple'].map(lambda x: x[0])
All_df['word_in_bullets_string_only_sum'] = All_df['word_in_bullets_string_only_tuple'].map(lambda x: x[1])
All_df['word_in_bullets_string_only_let'] = All_df['word_in_bullets_string_only_tuple'].map(lambda x: x[2])
All_df['word_in_bullets_string_only_numratio'] = All_df['word_in_bullets_string_only_tuple'].map(lambda x: x[3])
All_df['word_in_bullets_string_only_letratio'] = All_df['word_in_bullets_string_only_tuple'].map(lambda x: x[4])
All_df['word_in_bullets_string_only_string'] = All_df['word_in_bullets_string_only_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['word_in_bullets_string_only_tuple'],axis=1)

In [ ]:
All_df['word_in_bullets_w_dash']=All_df.apply(lambda x: \
            str_common_word(words_w_dash(x['search_term_stemmed']),words_w_dash(x['attribute_bullets_stemmed']))[0],axis=1)

All_df['two_words_in_bullets_tuple']=All_df.apply(lambda x: \
            str_2common_words_111(x['search_term_stemmed'],x['attribute_bullets_stemmed']),axis=1)
All_df['two_words_in_bullets_num'] = All_df['two_words_in_bullets_tuple'].map(lambda x: x[0])
All_df['two_words_in_bullets_sum'] = All_df['two_words_in_bullets_tuple'].map(lambda x: x[1])
All_df['two_words_in_bullets_let'] = All_df['two_words_in_bullets_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['two_words_in_bullets_tuple'],axis=1)

In [588]:
All_df['two_words_in_bullets_string_only_tuple']=All_df.apply(lambda x: \
            str_2common_words_111(x['search_term_stemmed'],x['attribute_bullets_stemmed'],string_only=True),axis=1)
All_df['two_words_in_bullets_string_only_num'] = All_df['two_words_in_bullets_string_only_tuple'].map(lambda x: x[0])
All_df['two_words_in_bullets_string_only_sum'] = All_df['two_words_in_bullets_string_only_tuple'].map(lambda x: x[1])
All_df['two_words_in_bullets_string_only_let'] = All_df['two_words_in_bullets_string_only_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['two_words_in_bullets_string_only_tuple'],axis=1)

In [589]:
All_df['common_digits_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_digits(x['search_term_stemmed'],x['attribute_bullets_stemmed']),axis=1)
All_df['len_of_digits_in_query'] = All_df['common_digits_in_bullets_tuple'].map(lambda x: x[0])
All_df['len_of_digits_in_bullets'] = All_df['common_digits_in_bullets_tuple'].map(lambda x: x[1])
All_df['common_digits_in_bullets_num'] = All_df['common_digits_in_bullets_tuple'].map(lambda x: x[2])
All_df['common_digits_in_bullets_ratio'] = All_df['common_digits_in_bullets_tuple'].map(lambda x: x[3])
All_df['common_digits_in_bullets_jaccard'] = All_df['common_digits_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['common_digits_in_bullets_tuple'],axis=1)

In [590]:
All_df['nn_important_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            x['attribute_bullets_stemmed']),axis=1)
All_df['nn_important_in_bullets_num'] = All_df['nn_important_in_bullets_tuple'].map(lambda x: x[0])
All_df['nn_important_in_bullets_sum'] = All_df['nn_important_in_bullets_tuple'].map(lambda x: x[1])
All_df['nn_important_in_bullets_let'] = All_df['nn_important_in_bullets_tuple'].map(lambda x: x[2])
All_df['nn_important_in_bullets_numratio'] = All_df['nn_important_in_bullets_tuple'].map(lambda x: x[3])
All_df['nn_important_in_bullets_letratio'] = All_df['nn_important_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_bullets_tuple'],axis=1)

In [591]:
All_df['nn_unimportant_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_unimportant_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            x['attribute_bullets_stemmed']),axis=1)
All_df['nn_unimportant_in_bullets_num'] = All_df['nn_unimportant_in_bullets_tuple'].map(lambda x: x[0])
All_df['nn_unimportant_in_bullets_let'] = All_df['nn_unimportant_in_bullets_tuple'].map(lambda x: x[2])
All_df['nn_unimportant_in_bullets_letratio'] = All_df['nn_unimportant_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_unimportant_in_bullets_tuple'],axis=1)

In [592]:
All_df['nn_important_in_nn_important_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_important_words(x['attribute_bullets_tokens']))),axis=1)
All_df['nn_important_in_nn_important_in_bullets_num'] = All_df['nn_important_in_nn_important_in_bullets_tuple'].map(lambda x: x[0])
All_df['nn_important_in_nn_important_in_bullets_sum'] = All_df['nn_important_in_nn_important_in_bullets_tuple'].map(lambda x: x[1])
All_df['nn_important_in_nn_important_in_bullets_let'] = All_df['nn_important_in_nn_important_in_bullets_tuple'].map(lambda x: x[2])
All_df['nn_important_in_nn_important_in_bullets_letratio'] = All_df['nn_important_in_nn_important_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_nn_important_in_bullets_tuple'],axis=1)

In [593]:
All_df['nn_important_in_nn_unimportant_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_important_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_unimportant_words(x['attribute_bullets_tokens']))),axis=1)
All_df['nn_important_in_nn_unimportant_in_bullets_num'] = All_df['nn_important_in_nn_unimportant_in_bullets_tuple'].map(lambda x: x[0])
All_df['nn_important_in_nn_unimportant_in_bullets_sum'] = All_df['nn_important_in_nn_unimportant_in_bullets_tuple'].map(lambda x: x[1])
All_df['nn_important_in_nn_unimportant_in_bullets_let'] = All_df['nn_important_in_nn_unimportant_in_bullets_tuple'].map(lambda x: x[2])
All_df['nn_important_in_nn_unimportant_in_bullets_letratio'] = All_df['nn_important_in_nn_unimportant_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_important_in_nn_unimportant_in_bullets_tuple'],axis=1)

In [594]:
All_df['nn_unimportant_in_nn_important_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(nn_unimportant_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(nn_important_words(x['attribute_bullets_tokens']))),axis=1)
All_df['nn_unimportant_in_nn_important_in_bullets_num'] = All_df['nn_unimportant_in_nn_important_in_bullets_tuple'].map(lambda x: x[0])
All_df['nn_unimportant_in_nn_important_in_bullets_sum'] = All_df['nn_unimportant_in_nn_important_in_bullets_tuple'].map(lambda x: x[1])
All_df['nn_unimportant_in_nn_important_in_bullets_let'] = All_df['nn_unimportant_in_nn_important_in_bullets_tuple'].map(lambda x: x[2])
All_df['nn_unimportant_in_nn_important_in_bullets_letratio'] = All_df['nn_unimportant_in_nn_important_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['nn_unimportant_in_nn_important_in_bullets_tuple'],axis=1)

In [595]:
All_df['jj_rb_in_jj_rb_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(jj_rb_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(jj_rb_words(x['attribute_bullets_tokens']))),axis=1)
All_df['jj_rb_in_jj_rb_in_bullets_num'] = All_df['jj_rb_in_jj_rb_in_bullets_tuple'].map(lambda x: x[0])
All_df['jj_rb_in_jj_rb_in_bullets_sum'] = All_df['jj_rb_in_jj_rb_in_bullets_tuple'].map(lambda x: x[1])
All_df['jj_rb_in_jj_rb_in_bullets_let'] = All_df['jj_rb_in_jj_rb_in_bullets_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['jj_rb_in_jj_rb_in_bullets_tuple'],axis=1)

In [596]:
All_df['vbg_in_vbg_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(str_stemmer_wo_parser(vbg_words(x['search_term_tokens']),stoplist=stoplist_wo_can),\
                            str_stemmer_wo_parser(vbg_words(x['attribute_bullets_tokens']))),axis=1)
All_df['vbg_in_vbg_in_bullets_num'] = All_df['vbg_in_vbg_in_bullets_tuple'].map(lambda x: x[0])
All_df['vbg_in_vbg_in_bullets_sum'] = All_df['vbg_in_vbg_in_bullets_tuple'].map(lambda x: x[1])
All_df['vbg_in_vbg_in_bullets_let'] = All_df['vbg_in_vbg_in_bullets_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['vbg_in_vbg_in_bullets_tuple'],axis=1)
print('words_in_bullets time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

words_in_bullets time: 16.7 minutes



In [597]:
t0 = time()

All_df['keyword_in_titlekeys_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_keys_stemmed'],x['product_title_keys_stemmed']),axis=1)
All_df['keyword_in_titlekeys_num'] = All_df['keyword_in_titlekeys_tuple'].map(lambda x: x[0])
All_df['keyword_in_titlekeys_sum'] = All_df['keyword_in_titlekeys_tuple'].map(lambda x: x[1])
All_df['keyword_in_titlekeys_let'] = All_df['keyword_in_titlekeys_tuple'].map(lambda x: x[2])
All_df['keyword_in_titlekeys_numratio'] = All_df['keyword_in_titlekeys_tuple'].map(lambda x: x[3])
All_df['keyword_in_titlekeys_letratio'] = All_df['keyword_in_titlekeys_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['keyword_in_titlekeys_tuple'],axis=1)

In [598]:
All_df['keyword_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_keys_stemmed'],x['product_description_stemmed']),axis=1)
All_df['keyword_in_description_num'] = All_df['keyword_in_description_tuple'].map(lambda x: x[0])
All_df['keyword_in_description_sum'] = All_df['keyword_in_description_tuple'].map(lambda x: x[1])
All_df['keyword_in_description_let'] = All_df['keyword_in_description_tuple'].map(lambda x: x[2])
All_df['keyword_in_description_numratio'] = All_df['keyword_in_description_tuple'].map(lambda x: x[3])
All_df['keyword_in_description_letratio'] = All_df['keyword_in_description_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['keyword_in_description_tuple'],axis=1)

In [599]:
All_df['keyword_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_keys_stemmed'],x['attribute_bullets_stemmed']),axis=1)
All_df['keyword_in_bullets_num'] = All_df['keyword_in_bullets_tuple'].map(lambda x: x[0])
All_df['keyword_in_bullets_sum'] = All_df['keyword_in_bullets_tuple'].map(lambda x: x[1])
All_df['keyword_in_bullets_let'] = All_df['keyword_in_bullets_tuple'].map(lambda x: x[2])
All_df['keyword_in_bullets_numratio'] = All_df['keyword_in_bullets_tuple'].map(lambda x: x[3])
All_df['keyword_in_bullets_letratio'] = All_df['keyword_in_bullets_tuple'].map(lambda x: x[4])
All_df=All_df.drop(['keyword_in_bullets_tuple'],axis=1)

In [600]:
All_df['jaccard_keyword_in_titlekeys_tuple']=All_df.apply(lambda x: \
            str_jaccard(x['search_term_keys_stemmed'],x['product_title_keys_stemmed']),axis=1)
All_df['keyword_in_titlekeys_jacnum'] = All_df['jaccard_keyword_in_titlekeys_tuple'].map(lambda x: x[0])
All_df['keyword_in_titlekeys_jaclet'] = All_df['jaccard_keyword_in_titlekeys_tuple'].map(lambda x: x[1])
All_df=All_df.drop(['jaccard_keyword_in_titlekeys_tuple'],axis=1)

In [601]:
def concatenate_keys(beforethekey,thekey):
    l=[beforethekey,thekey]
    if "" in l:
        l.remove("")
    if "" in l:
        l.remove("")
    return " ".join(l)

In [602]:
All_df['thekeys_in_title']=All_df.apply(lambda x: \
            query_in_text(concatenate_keys(x['search_term_beforethekey_stemmed'],x['search_term_thekey_stemmed']),\
                          x['product_title_stemmed']), axis=1)  
All_df['thekeys_in_description']=All_df.apply(lambda x: \
            query_in_text(concatenate_keys(x['search_term_beforethekey_stemmed'],x['search_term_thekey_stemmed']),\
                          x['product_description_stemmed']), axis=1)  
All_df['thekeys_in_bullets']=All_df.apply(lambda x: \
            query_in_text(concatenate_keys(x['search_term_beforethekey_stemmed'],x['search_term_thekey_stemmed']),\
                          x['attribute_bullets_stemmed']), axis=1)                            

All_df['thekey_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_thekey_stemmed'],x['product_description_stemmed']), axis=1) 
All_df['thekey_in_description_sum'] = All_df['thekey_in_description_tuple'].map(lambda x: x[1])
All_df['thekey_in_description_let'] = All_df['thekey_in_description_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['thekey_in_description_tuple'],axis=1)

All_df['thekey_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_thekey_stemmed'],x['attribute_bullets_stemmed']), axis=1) 
All_df['thekey_in_bullets_sum'] = All_df['thekey_in_bullets_tuple'].map(lambda x: x[1])
All_df['thekey_in_bullets_let'] = All_df['thekey_in_bullets_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['thekey_in_bullets_tuple'],axis=1)


All_df['beforethekey_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_beforethekey_stemmed'],x['product_description_stemmed']), axis=1) 
All_df['beforethekey_in_description_sum'] = All_df['beforethekey_in_description_tuple'].map(lambda x: x[1])
All_df['beforethekey_in_description_let'] = All_df['beforethekey_in_description_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['beforethekey_in_description_tuple'],axis=1)

All_df['beforethekey_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_beforethekey_stemmed'],x['attribute_bullets_stemmed']), axis=1) 
All_df['beforethekey_in_bullets_sum'] = All_df['beforethekey_in_bullets_tuple'].map(lambda x: x[1])
All_df['beforethekey_in_bullets_let'] = All_df['beforethekey_in_bullets_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['beforethekey_in_bullets_tuple'],axis=1)

In [604]:
All_df['thekey_in_nn_important_in_description_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_thekey_stemmed'],
                            str_stemmer_wo_parser(nn_important_words(str(x['product_description_tokens'])))),axis=1)
All_df['thekey_in_nn_important_in_description_sum'] = All_df['thekey_in_nn_important_in_description_tuple'].map(lambda x: x[1])
All_df['thekey_in_nn_important_in_description_let'] = All_df['thekey_in_nn_important_in_description_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['thekey_in_nn_important_in_description_tuple'],axis=1)

All_df['thekey_in_nn_important_in_bullets_tuple']=All_df.apply(lambda x: \
            str_common_word(x['search_term_thekey_stemmed'],\
                            str_stemmer_wo_parser(nn_important_words(x['attribute_bullets_tokens']))),axis=1)
All_df['thekey_in_nn_important_in_bullets_sum'] = All_df['thekey_in_nn_important_in_bullets_tuple'].map(lambda x: x[1])
All_df['thekey_in_nn_important_in_bullets_let'] = All_df['thekey_in_nn_important_in_bullets_tuple'].map(lambda x: x[2])
All_df=All_df.drop(['thekey_in_nn_important_in_bullets_tuple'],axis=1)


In [605]:
def match_thekeywords(word1,word2):
    return int(len(word1)>0 and len(word2)>0 and (word1 in word2 or word2 in word1))

All_df['thekey_in_thekey']=All_df.apply(lambda x: \
            match_thekeywords(x['search_term_thekey_stemmed'],x['product_title_thekey_stemmed']),axis=1)
All_df['beforethekey_in_beforethekey']=All_df.apply(lambda x: \
            match_thekeywords(x['search_term_beforethekey_stemmed'],x['product_title_beforethekey_stemmed']),axis=1) 
All_df['beforethekeys_in_beforethekeys']=All_df.apply(lambda x: \
            max(match_thekeywords(x['search_term_beforethekey_stemmed'],x['product_title_beforethekey_stemmed']),\
                match_thekeywords(x['search_term_beforethekey_stemmed'],x['product_title_before2thekey_stemmed']),\
                match_thekeywords(x['search_term_before2thekey_stemmed'],x['product_title_beforethekey_stemmed']),\
                match_thekeywords(x['search_term_before2thekey_stemmed'],x['product_title_before2thekey_stemmed'])),axis=1)
All_df['thekey_in_beforethekeys']=All_df.apply(lambda x: \
            max(match_thekeywords(x['search_term_thekey_stemmed'],x['product_title_beforethekey_stemmed']),\
                match_thekeywords(x['search_term_thekey_stemmed'],x['product_title_before2thekey_stemmed'])),axis=1)
All_df['beforethekeys_in_thekey']=All_df.apply(lambda x: \
            max(match_thekeywords(x['search_term_thekey_stemmed'],x['product_title_beforethekey_stemmed']),\
                match_thekeywords(x['search_term_thekey_stemmed'],x['product_title_before2thekey_stemmed'])),axis=1)

In [606]:
### Use NLTK WordNet to calculate similarities between keywords
t2 = time()

##############
All_df['key_for_dict']=All_df.apply(lambda x: x['search_term_thekey']+"\t"+x['product_title_thekey'],axis=1) 
aa=list(set(list(All_df['key_for_dict'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=find_similarity(aa[i].split("\t")[0],aa[i].split("\t")[1])
    if (i % 5000)==0:
        print(""+str(i)+" out of "+str(len(aa))+" unique combinations; "+str(round((time()-t2)/60,1))+" minutes")
    
All_df['thekeys_similarity_tuple']=All_df['key_for_dict'].map(lambda x: my_dict[x] )
All_df['thekeys_pathsimilarity_max'] = All_df['thekeys_similarity_tuple'].map(lambda x: x[0])
All_df['thekeys_pathsimilarity_mean'] = All_df['thekeys_similarity_tuple'].map(lambda x: x[1])
All_df['thekeys_lchsimilarity_max'] = All_df['thekeys_similarity_tuple'].map(lambda x: x[2])
All_df['thekeys_lchsimilarity_mean'] = All_df['thekeys_similarity_tuple'].map(lambda x: x[3])
All_df['thekeys_ressimilarity_max'] = All_df['thekeys_similarity_tuple'].map(lambda x: x[4])
All_df['thekeys_ressimilarity_mean'] = All_df['thekeys_similarity_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['thekeys_similarity_tuple'],axis=1)
print('thekeys similarity time:',round((time()-t2)/60,1) ,'minutes\n')
t2 = time()

0 out of 39278 unique combinations; 0.3 minutes
5000 out of 39278 unique combinations; 1.2 minutes
10000 out of 39278 unique combinations; 2.2 minutes
15000 out of 39278 unique combinations; 3.1 minutes
20000 out of 39278 unique combinations; 4.1 minutes
25000 out of 39278 unique combinations; 5.0 minutes
30000 out of 39278 unique combinations; 6.0 minutes
35000 out of 39278 unique combinations; 6.9 minutes
thekeys similarity time: 7.8 minutes



In [607]:
All_df['key_for_dict']=All_df.apply(lambda x: x['search_term_beforethekey']+"\t"+x['product_title_beforethekey'],axis=1) 
aa=list(set(list(All_df['key_for_dict'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=find_similarity(aa[i].split("\t")[0],aa[i].split("\t")[1],nouns=False)
    if (i % 5000)==0:
        print(""+str(i)+" out of "+str(len(aa))+" unique combinations; "+str(round((time()-t2)/60,1))+" minutes")
    
All_df['beforethekeys_similarity_tuple']=All_df['key_for_dict'].map(lambda x: my_dict[x] )
All_df['beforethekeys_pathsimilarity_max'] = All_df['beforethekeys_similarity_tuple'].map(lambda x: x[0])
All_df['beforethekeys_pathsimilarity_mean'] = All_df['beforethekeys_similarity_tuple'].map(lambda x: x[1])
All_df['beforethekeys_lchsimilarity_max'] = All_df['beforethekeys_similarity_tuple'].map(lambda x: x[2])
All_df['beforethekeys_lchsimilarity_mean'] = All_df['beforethekeys_similarity_tuple'].map(lambda x: x[3])
All_df['beforethekeys_ressimilarity_max'] = All_df['beforethekeys_similarity_tuple'].map(lambda x: x[4])
All_df['beforethekeys_ressimilarity_mean'] = All_df['beforethekeys_similarity_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['beforethekeys_similarity_tuple'],axis=1)
print('beforethekeys similarity time:',round((time()-t2)/60,1) ,'minutes\n')
t2 = time()

0 out of 43176 unique combinations; 0.3 minutes
5000 out of 43176 unique combinations; 1.6 minutes
10000 out of 43176 unique combinations; 2.8 minutes
15000 out of 43176 unique combinations; 4.1 minutes
20000 out of 43176 unique combinations; 5.2 minutes
25000 out of 43176 unique combinations; 6.4 minutes
30000 out of 43176 unique combinations; 7.6 minutes
35000 out of 43176 unique combinations; 8.8 minutes
40000 out of 43176 unique combinations; 10.0 minutes
beforethekeys similarity time: 10.8 minutes



In [608]:
All_df['key_for_dict']=All_df.apply(lambda x: x['search_term_thekey']+"\t"+x['product_title_beforethekey'],axis=1) 
aa=list(set(list(All_df['key_for_dict'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=find_similarity(aa[i].split("\t")[0],aa[i].split("\t")[1],nouns=False)
    if (i % 5000)==0:
        print(""+str(i)+" out of "+str(len(aa))+" unique combinations; "+str(round((time()-t2)/60,1))+" minutes")
All_df['thekey_beforethekey_similarity_tuple']=All_df['key_for_dict'].map(lambda x: my_dict[x] )
All_df['thekey_beforethekey_pathsimilarity_max'] = All_df['thekey_beforethekey_similarity_tuple'].map(lambda x: x[0])
All_df['thekey_beforethekey_pathsimilarity_mean'] = All_df['thekey_beforethekey_similarity_tuple'].map(lambda x: x[1])
All_df['thekey_beforethekey_lchsimilarity_max'] = All_df['thekey_beforethekey_similarity_tuple'].map(lambda x: x[2])
All_df['thekey_beforethekey_lchsimilarity_mean'] = All_df['thekey_beforethekey_similarity_tuple'].map(lambda x: x[3])
All_df['thekey_beforethekey_ressimilarity_max'] = All_df['thekey_beforethekey_similarity_tuple'].map(lambda x: x[4])
All_df['thekey_beforethekey_ressimilarity_mean'] = All_df['thekey_beforethekey_similarity_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['thekey_beforethekey_similarity_tuple'],axis=1)
print('thekey_beforethekey similarity time:',round((time()-t2)/60,1) ,'minutes\n')
t2 = time()

0 out of 52374 unique combinations; 0.3 minutes
5000 out of 52374 unique combinations; 1.7 minutes
10000 out of 52374 unique combinations; 3.0 minutes
15000 out of 52374 unique combinations; 4.4 minutes
20000 out of 52374 unique combinations; 5.8 minutes
25000 out of 52374 unique combinations; 7.1 minutes
30000 out of 52374 unique combinations; 8.5 minutes
35000 out of 52374 unique combinations; 9.9 minutes
40000 out of 52374 unique combinations; 11.2 minutes
45000 out of 52374 unique combinations; 12.6 minutes
50000 out of 52374 unique combinations; 13.9 minutes
thekey_beforethekey similarity time: 14.6 minutes



In [609]:
All_df['key_for_dict']=All_df.apply(lambda x: x['search_term_thekey']+"\t"+x['product_title_before2thekey'],axis=1) 
aa=list(set(list(All_df['key_for_dict'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=find_similarity(aa[i].split("\t")[0],aa[i].split("\t")[1],nouns=False)
    if (i % 5000)==0:
        print(""+str(i)+" out of "+str(len(aa))+" unique combinations; "+str(round((time()-t2)/60,1))+" minutes")
All_df['thekey_before2thekey_similarity_tuple']=All_df['key_for_dict'].map(lambda x: my_dict[x] )
All_df['thekey_before2thekey_pathsimilarity_max'] = All_df['thekey_before2thekey_similarity_tuple'].map(lambda x: x[0])
All_df['thekey_before2thekey_pathsimilarity_mean'] = All_df['thekey_before2thekey_similarity_tuple'].map(lambda x: x[1])
All_df['thekey_before2thekey_lchsimilarity_max'] = All_df['thekey_before2thekey_similarity_tuple'].map(lambda x: x[2])
All_df['thekey_before2thekey_lchsimilarity_mean'] = All_df['thekey_before2thekey_similarity_tuple'].map(lambda x: x[3])
All_df['thekey_before2thekey_ressimilarity_max'] = All_df['thekey_before2thekey_similarity_tuple'].map(lambda x: x[4])
All_df['thekey_before2thekey_ressimilarity_mean'] = All_df['thekey_before2thekey_similarity_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['thekey_before2thekey_similarity_tuple'],axis=1)
print('thekey_before2thekey similarity time:',round((time()-t2)/60,1) ,'minutes\n')
t2 = time()

for var_name in ['pathsimilarity_max','pathsimilarity_mean','lchsimilarity_max','lchsimilarity_mean',
                 'ressimilarity_max', 'ressimilarity_mean']:
    All_df['thekey_beforethekeys_'+var_name] = All_df.apply(lambda x: \
        max(x['thekey_beforethekey_'+var_name], x['thekey_before2thekey_'+var_name]),axis=1)

0 out of 42371 unique combinations; 0.3 minutes
5000 out of 42371 unique combinations; 1.5 minutes
10000 out of 42371 unique combinations; 2.8 minutes
15000 out of 42371 unique combinations; 4.0 minutes
20000 out of 42371 unique combinations; 5.3 minutes
25000 out of 42371 unique combinations; 6.6 minutes
30000 out of 42371 unique combinations; 7.8 minutes
35000 out of 42371 unique combinations; 9.1 minutes
40000 out of 42371 unique combinations; 10.4 minutes
thekey_before2thekey similarity time: 11.0 minutes



In [610]:
All_df['key_for_dict']=All_df.apply(lambda x: x['search_term_beforethekey']+"\t"+x['product_title_thekey'],axis=1) 
aa=list(set(list(All_df['key_for_dict'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=find_similarity(aa[i].split("\t")[0],aa[i].split("\t")[1],nouns=False)
    if (i % 5000)==0:
        print(""+str(i)+" out of "+str(len(aa))+" unique combinations; "+str(round((time()-t2)/60,1))+" minutes")
    
All_df['beforethekey_thekey_similarity_tuple']=All_df['key_for_dict'].map(lambda x: my_dict[x] )
All_df['beforethekey_thekey_pathsimilarity_max'] = All_df['beforethekey_thekey_similarity_tuple'].map(lambda x: x[0])
All_df['beforethekey_thekey_pathsimilarity_mean'] = All_df['beforethekey_thekey_similarity_tuple'].map(lambda x: x[1])
All_df['beforethekey_thekey_lchsimilarity_max'] = All_df['beforethekey_thekey_similarity_tuple'].map(lambda x: x[2])
All_df['beforethekey_thekey_lchsimilarity_mean'] = All_df['beforethekey_thekey_similarity_tuple'].map(lambda x: x[3])
All_df['beforethekey_thekey_ressimilarity_max'] = All_df['beforethekey_thekey_similarity_tuple'].map(lambda x: x[4])
All_df['beforethekey_thekey_ressimilarity_mean'] = All_df['beforethekey_thekey_similarity_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['beforethekey_thekey_similarity_tuple'],axis=1)
print('beforethekey_thekey similarity time:',round((time()-t2)/60,1) ,'minutes\n')
t2 = time()

0 out of 33754 unique combinations; 2.1 minutes
5000 out of 33754 unique combinations; 3.5 minutes
10000 out of 33754 unique combinations; 4.9 minutes
15000 out of 33754 unique combinations; 6.3 minutes
20000 out of 33754 unique combinations; 7.6 minutes
25000 out of 33754 unique combinations; 9.0 minutes
30000 out of 33754 unique combinations; 10.4 minutes
beforethekey_thekey similarity time: 11.5 minutes



In [616]:
All_df.to_csv('maybe_final.csv')

In [618]:
All_df['key_for_dict']=All_df.apply(lambda x: x['search_term_thekey']+"\t"+x['search_term_beforethekey'],axis=1) 
aa=list(set(list(All_df['key_for_dict'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=find_similarity(aa[i].split("\t")[0],aa[i].split("\t")[1],nouns=False)
    if (i % 5000)==0:
        print(""+str(i)+" out of "+str(len(aa))+" unique combinations; "+str(round((time()-t2)/60,1))+" minutes")
    
All_df['query_similarity_tuple']=All_df['key_for_dict'].map(lambda x: my_dict[x] )
All_df['query_pathsimilarity_max'] = All_df['query_similarity_tuple'].map(lambda x: x[0])
All_df['query_pathsimilarity_mean'] = All_df['query_similarity_tuple'].map(lambda x: x[1])
All_df['query_lchhsimilarity_max'] = All_df['query_similarity_tuple'].map(lambda x: x[2])
All_df['query_lchsimilarity_mean'] = All_df['query_similarity_tuple'].map(lambda x: x[3])
All_df['query_ressimilarity_max'] = All_df['query_similarity_tuple'].map(lambda x: x[4])
All_df['query_ressimilarity_mean'] = All_df['query_similarity_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['query_similarity_tuple'],axis=1)
print('query similarity time:',round((time()-t2)/60,1) ,'minutes\n')
t2 = time()

0 out of 13945 unique combinations; 809.7 minutes
5000 out of 13945 unique combinations; 811.0 minutes
10000 out of 13945 unique combinations; 812.3 minutes
query similarity time: 813.3 minutes



In [619]:
All_df['key_for_dict']=All_df.apply(lambda x: x['product_title_thekey']+"\t"+x['product_title_beforethekey'],axis=1) 
aa=list(set(list(All_df['key_for_dict'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=find_similarity(aa[i].split("\t")[0],aa[i].split("\t")[1],nouns=False)
    if (i % 5000)==0:
        print(""+str(i)+" out of "+str(len(aa))+" unique combinations; "+str(round((time()-t2)/60,1))+" minutes")
    
All_df['title_similarity_tuple']=All_df['key_for_dict'].map(lambda x: my_dict[x] )
All_df['title_pathsimilarity_max'] = All_df['title_similarity_tuple'].map(lambda x: x[0])
All_df['title_pathsimilarity_mean'] = All_df['title_similarity_tuple'].map(lambda x: x[1])
All_df['title_lchsimilarity_max'] = All_df['title_similarity_tuple'].map(lambda x: x[2])
All_df['title_lchsimilarity_mean'] = All_df['title_similarity_tuple'].map(lambda x: x[3])
All_df['title_ressimilarity_max'] = All_df['title_similarity_tuple'].map(lambda x: x[4])
All_df['title_ressimilarity_mean'] = All_df['title_similarity_tuple'].map(lambda x: x[5])
All_df=All_df.drop(['title_similarity_tuple'],axis=1)
print('title similarity time:',round((time()-t2)/60,1) ,'minutes\n')
t2 = time()


0 out of 22526 unique combinations; 1.3 minutes
5000 out of 22526 unique combinations; 2.6 minutes
10000 out of 22526 unique combinations; 4.0 minutes
15000 out of 22526 unique combinations; 5.4 minutes
20000 out of 22526 unique combinations; 6.7 minutes
title similarity time: 7.4 minutes



In [620]:
All_df['seqmatch_title_tuple']=All_df.apply(lambda x: \
            seq_matcher(x['search_term_stemmed'],x['product_title_stemmed']),axis=1)
All_df['seqmatch_title_ratio'] = All_df['seqmatch_title_tuple'].map(lambda x: x[0])
All_df['seqmatch_title_ratioscaled'] = All_df['seqmatch_title_tuple'].map(lambda x: x[1])
All_df=All_df.drop(['seqmatch_title_tuple'],axis=1)


All_df['seqmatch_description_tuple']=All_df.apply(lambda x: \
            seq_matcher(x['search_term_stemmed'],x['product_description_stemmed']),axis=1)
All_df['seqmatch_description_ratio'] = All_df['seqmatch_description_tuple'].map(lambda x: x[0])
All_df['seqmatch_description_ratioscaled'] = All_df['seqmatch_description_tuple'].map(lambda x: x[1])
All_df=All_df.drop(['seqmatch_description_tuple'],axis=1)

All_df['seqmatch_bullets_tuple']=All_df.apply(lambda x: \
            seq_matcher(x['search_term_stemmed'],x['attribute_bullets_stemmed']),axis=1)
All_df['seqmatch_bullets_ratio'] = All_df['seqmatch_bullets_tuple'].map(lambda x: x[0])
All_df['seqmatch_bullets_ratioscaled'] = All_df['seqmatch_bullets_tuple'].map(lambda x: x[1])
All_df=All_df.drop(['seqmatch_bullets_tuple'],axis=1)

All_df['seqmatch_desc&bullets_tuple']=All_df.apply(lambda x: \
            seq_matcher(x['search_term_stemmed'],x['product_description_stemmed']+" "+x['attribute_bullets_stemmed']),axis=1)
All_df['seqmatch_desc&bullets_ratio'] = All_df['seqmatch_desc&bullets_tuple'].map(lambda x: x[0])
All_df['seqmatch_desc&bullets_ratioscaled'] = All_df['seqmatch_desc&bullets_tuple'].map(lambda x: x[1])
All_df=All_df.drop(['seqmatch_desc&bullets_tuple'],axis=1)

print('sequence match time:',round((time()-t0)/60,1) ,'minutes\n')

sequence match time: 892.6 minutes



In [621]:
### STEP 7: Some TFIDF features
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from scipy.sparse import csr_matrix

vectorizer_title =  TfidfVectorizer(stop_words='english',max_df=0.5) 
vectorizer_description =  TfidfVectorizer(stop_words='english',max_df=0.5) 
vectorizer_bullets =  TfidfVectorizer(stop_words='english',max_df=0.5) 

features_title = vectorizer_title.fit_transform(list(set(list(All_df['product_title_stemmed'])))) 
features_description = vectorizer_description.fit_transform(list(set(list(All_df['product_description_stemmed'])))) 
features_bullets = vectorizer_bullets.fit_transform(list(set(list(All_df['attribute_bullets_stemmed'])))) 

tfidf_title = vectorizer_title.transform(All_df['search_term_stemmed']) 
tfidf_description = vectorizer_description.transform(All_df['search_term_stemmed']) 
tfidf_bullets = vectorizer_bullets.transform(All_df['search_term_stemmed']) 

tfidf_title_querythekey = vectorizer_title.transform(All_df['search_term_thekey_stemmed'])
tfidf_title_querybeforethekey = vectorizer_title.transform(All_df['search_term_beforethekey_stemmed'])


tfidf_matchtitle = vectorizer_title.transform(All_df['word_in_title_string'])
tfidf_matchtitle_stringonly = vectorizer_title.transform(All_df['word_in_title_string_only_string'])
tfidf_matchdescription = vectorizer_description.transform(All_df['word_in_description_string']) 
tfidf_matchdescription_stringonly = vectorizer_description.transform(All_df['word_in_description_string_only_string']) 
tfidf_matchbullets = vectorizer_bullets.transform(All_df['word_in_bullets_string']) 
tfidf_matchbullets_stringonly = vectorizer_bullets.transform(All_df['word_in_bullets_string_only_string']) 

len(vectorizer_title.get_feature_names())
len(vectorizer_description.get_feature_names())
len(vectorizer_bullets.get_feature_names())

uno_title=np.ones((len(vectorizer_title.get_feature_names()),1))
uno_description=np.ones((len(vectorizer_description.get_feature_names()),1))
uno_bullets=np.ones((len(vectorizer_bullets.get_feature_names()),1))

let_title=np.asarray([[len(word)] for word in vectorizer_title.get_feature_names()])
let_description=np.asarray([[len(word)] for word in vectorizer_description.get_feature_names()])
let_bullets=np.asarray([[len(word)] for word in vectorizer_bullets.get_feature_names()])




All_df['tfidf_title_num']=tfidf_title.tocsr().dot(uno_title)
All_df['tfidf_description_num']=tfidf_description.tocsr().dot(uno_description)
All_df['tfidf_bullets_num']=tfidf_bullets.tocsr().dot(uno_bullets)

All_df['tfidf_title_let']=tfidf_title.tocsr().dot(let_title)
All_df['tfidf_description_let']=tfidf_description.tocsr().dot(let_description)
All_df['tfidf_bullets_let']=tfidf_bullets.tocsr().dot(let_bullets)

All_df['tfidf_matchtitle_num']=tfidf_matchtitle.tocsr().dot(uno_title)
All_df['tfidf_matchdescription_num']=tfidf_matchdescription.tocsr().dot(uno_description)
All_df['tfidf_matchbullets_num']=tfidf_matchbullets.tocsr().dot(uno_bullets)

All_df['tfidf_matchtitle_stringonly_num']=tfidf_matchtitle_stringonly.tocsr().dot(uno_title)
All_df['tfidf_matchdescription_stringonly_num']=tfidf_matchdescription_stringonly.tocsr().dot(uno_description)
All_df['tfidf_matchbullets_stringonly_num']=tfidf_matchbullets_stringonly.tocsr().dot(uno_bullets)


All_df['tfidf_title_querythekey_num']=tfidf_title_querythekey.tocsr().dot(uno_title)
All_df['tfidf_title_querybeforethekey_num']=tfidf_title_querybeforethekey.tocsr().dot(uno_title)
All_df['tfidf_title_querythekey_let']=tfidf_title_querythekey.tocsr().dot(let_title)
All_df['tfidf_title_querybeforethekey_let']=tfidf_title_querybeforethekey.tocsr().dot(let_title)

In [622]:
print('tfidf basic features time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()

All_df['tfidf_nn_important_in_title_num']=vectorizer_title.transform(All_df['search_term_tokens'].map(lambda x:nn_important_words(x)) ).tocsr().dot(uno_title)
All_df['tfidf_nn_important_in_description_num']=vectorizer_description.transform(All_df['search_term_tokens'].map(lambda x:nn_important_words(x)) ).tocsr().dot(uno_description)
All_df['tfidf_nn_important_in_bullets_num']=vectorizer_bullets.transform(All_df['search_term_tokens'].map(lambda x:nn_important_words(x)) ).tocsr().dot(uno_bullets)
All_df['tfidf_nn_important_in_title_let']=vectorizer_title.transform(All_df['search_term_tokens'].map(lambda x:nn_important_words(x)) ).tocsr().dot(let_title)
All_df['tfidf_nn_important_in_description_let']=vectorizer_description.transform(All_df['search_term_tokens'].map(lambda x:nn_important_words(x)) ).tocsr().dot(let_description)
All_df['tfidf_nn_important_in_bullets_let']=vectorizer_bullets.transform(All_df['search_term_tokens'].map(lambda x:nn_important_words(x)) ).tocsr().dot(let_bullets)

All_df['tfidf_nn_unimportant_in_title_num']=vectorizer_title.transform(All_df['search_term_tokens'].map(lambda x:nn_unimportant_words(x)) ).tocsr().dot(uno_title)
All_df['tfidf_nn_unimportant_in_description_num']=vectorizer_description.transform(All_df['search_term_tokens'].map(lambda x:nn_unimportant_words(x)) ).tocsr().dot(uno_description)
All_df['tfidf_nn_unimportant_in_bullets_num']=vectorizer_bullets.transform(All_df['search_term_tokens'].map(lambda x:nn_unimportant_words(x)) ).tocsr().dot(uno_bullets)
All_df['tfidf_nn_unimportant_in_title_let']=vectorizer_title.transform(All_df['search_term_tokens'].map(lambda x:nn_unimportant_words(x)) ).tocsr().dot(let_title)
All_df['tfidf_nn_unimportant_in_description_let']=vectorizer_description.transform(All_df['search_term_tokens'].map(lambda x:nn_unimportant_words(x)) ).tocsr().dot(let_description)
All_df['tfidf_nn_unimportant_in_bullets_let']=vectorizer_bullets.transform(All_df['search_term_tokens'].map(lambda x:nn_unimportant_words(x)) ).tocsr().dot(let_bullets)

All_df['tfidf_vbg_in_title_num']=vectorizer_title.transform(All_df['search_term_tokens'].map(lambda x:vbg_words(x)) ).tocsr().dot(uno_title)
All_df['tfidf_vbg_in_description_num']=vectorizer_description.transform(All_df['search_term_tokens'].map(lambda x:vbg_words(x)) ).tocsr().dot(uno_description)
All_df['tfidf_vbg_in_bullets_num']=vectorizer_bullets.transform(All_df['search_term_tokens'].map(lambda x:vbg_words(x)) ).tocsr().dot(uno_bullets)
All_df['tfidf_vbg_in_title_let']=vectorizer_title.transform(All_df['search_term_tokens'].map(lambda x:vbg_words(x)) ).tocsr().dot(let_title)
All_df['tfidf_vbg_in_description_let']=vectorizer_description.transform(All_df['search_term_tokens'].map(lambda x:vbg_words(x)) ).tocsr().dot(let_description)
All_df['tfidf_vbg_in_bullets_let']=vectorizer_bullets.transform(All_df['search_term_tokens'].map(lambda x:vbg_words(x)) ).tocsr().dot(let_bullets)


All_df['tfidf_jj_rb_in_title_num']=vectorizer_title.transform(All_df['search_term_tokens'].map(lambda x:jj_rb_words(x)) ).tocsr().dot(uno_title)
All_df['tfidf_jj_rb_in_description_num']=vectorizer_description.transform(All_df['search_term_tokens'].map(lambda x:jj_rb_words(x)) ).tocsr().dot(uno_description)
All_df['tfidf_jj_rb_in_bullets_num']=vectorizer_bullets.transform(All_df['search_term_tokens'].map(lambda x:jj_rb_words(x)) ).tocsr().dot(uno_bullets)
All_df['tfidf_jj_rb_in_title_let']=vectorizer_title.transform(All_df['search_term_tokens'].map(lambda x:jj_rb_words(x)) ).tocsr().dot(let_title)
All_df['tfidf_jj_rb_in_description_let']=vectorizer_description.transform(All_df['search_term_tokens'].map(lambda x:jj_rb_words(x)) ).tocsr().dot(let_description)
All_df['tfidf_jj_rb_in_bullets_let']=vectorizer_bullets.transform(All_df['search_term_tokens'].map(lambda x:jj_rb_words(x)) ).tocsr().dot(let_bullets)



All_df=All_df.drop(['word_in_title_string','word_in_title_string_only_string','word_in_description_string',\
'word_in_description_string_only_string','word_in_bullets_string','word_in_bullets_string_only_string'],axis=1)

print('tfidf advanced features time:',round((time()-t0)/60,1) ,'minutes\n')
t0 = time()


tfidf basic features time: 894.3 minutes

tfidf advanced features time: 1.9 minutes



In [636]:
All_df.to_csv('cleaning_data.csv')

In [638]:
list(All_df.columns)

['id',
 'product_title',
 'product_uid',
 'relevance',
 'search_term',
 'product_description',
 'Brand',
 'Certification',
 'Color',
 'Bullet',
 'Product Height (in.)',
 'Product Width (in.)',
 'Product Depth (in.)',
 'Product Weight (lb.)',
 'Product Length (in.)',
 'Indoor/Outdoor',
 'Commercial / Residential',
 'ENERGY STAR Certified',
 'Package Quantity',
 'Hardware Included',
 'product_title_simpleparsed',
 'search_term_simpleparsed',
 'search_term_parsed',
 'search_term_parsed_wospellcheck',
 'product_title_parsed',
 'brand_parsed',
 'Material',
 'material_parsed',
 'search_term_parsed_woBrand',
 'brands_in_search_term',
 'search_term_parsed_woBM',
 'materials_in_search_term',
 'product_title_parsed_woBrand',
 'brands_in_product_title',
 'product_title_parsed_woBM',
 'materials_in_product_title',
 'search_term_tokens',
 'product_title_tokens',
 'attribute_bullets_parsed',
 'search_term_for',
 'search_term_for_stemmed',
 'search_term_with',
 'search_term_with_stemmed',
 'product_t